In [54]:
import pandas as pd
import numpy as np
from statsmodels.regression.rolling import RollingOLS
from scipy import stats
import statsmodels.regression.linear_model as sm
import statsmodels.tools.tools as sm2
from statsmodels.regression.linear_model import OLS
#from pyfinance import PandasRollingOLS
from statsmodels.api import add_constant

In [55]:
#read in ffdata
new_row = pd.DataFrame({'-0.000800':-0.000800, '0.008300':0.008300, '0.004100':0.004100, 
                        '0.000040':0.000040}, index = [0])
ffdata = pd.read_csv("ffdata.txt", delim_whitespace=True)
ffdata = pd.concat([new_row, ffdata]).reset_index(drop = True) 
ffdata.rename(columns={'-0.000800':'Market Returns', '0.008300':'Returns to the Fama-French size factor', 
                       '0.004100':'Returns to the Fama-French value factor', 
                       '0.000040':'Risk-free rate'}, inplace = True)
ffdata

,Market Returns,Returns to the Fama-French size factor,Returns to the Fama-French value factor,Risk-free rate
0,-0.0008,0.0083,0.0041,0.00004
1,0.0122,0.0035,0.0002,0.00004
2,0.0019,0.0012,0.0018,0.00004
3,0.0027,0.0050,-0.0007,0.00004
4,0.0048,0.0033,0.0062,0.00004
...,...,...,...,...
1506,0.0052,-0.0008,0.0016,0.00000
1507,0.0010,-0.0015,-0.0030,0.00000
1508,-0.0009,0.0008,-0.0008,0.00000
1509,-0.0007,-0.0004,-0.0018,0.00000


In [56]:
#load in ticker.txt - tickers
ticker = pd.read_csv('ticker.txt', header=None)
ticker = ticker.reset_index()
ticker['index'] = ticker['index'] + 1
ticker

,index,0
0,1,A
1,2,AA
2,3,AAI
3,4,AAON
4,5,AAP
...,...,...
1872,1873,ZMH
1873,1874,ZOLL
1874,1875,ZOLT
1875,1876,ZQK


In [204]:
#load in retdate.txt - return dates
retdate = pd.read_csv('retdate.txt', sep = " ", header = None)
retdate.loc[1510]

0    20091231
Name: 1510, dtype: int64

In [58]:
#load in secdata.txt - securities data
secdata = pd.read_csv('secdata.txt', sep = " ", header = None)
secdata.columns = ['Ticker #', 'Stock Returns', 'Market Capitalizations']
secdata

,Ticker #,Stock Returns,Market Capitalizations
0,1,-0.015048,13713091.20
1,1,0.026042,14070202.95
2,1,0.031472,14513021.52
3,1,0.012795,14698719.63
4,1,0.045675,15370089.72
...,...,...,...
2836142,1877,0.000000,580951.00
2836143,1877,-0.008079,576257.50
2836144,1877,0.000000,576257.50
2836145,1877,0.008145,580951.00


In [59]:
#merge ticker with stocks
stock_with_ticker = secdata.merge(ticker, left_on = "Ticker #", right_on = "index")
stock_with_ticker.to_csv('stock_with_ticker.csv')

# Question 2

We now consider the modern portfolio theory (MPT) approach to estimating volatility. Each step below should be completed using 504, 252, 126, and 63 day rolling windows.

## a) Pick a portfolio of 100 securities.

Criteria: 1st 100 securities.

In [60]:
secdata

,Ticker #,Stock Returns,Market Capitalizations
0,1,-0.015048,13713091.20
1,1,0.026042,14070202.95
2,1,0.031472,14513021.52
3,1,0.012795,14698719.63
4,1,0.045675,15370089.72
...,...,...,...
2836142,1877,0.000000,580951.00
2836143,1877,-0.008079,576257.50
2836144,1877,0.000000,576257.50
2836145,1877,0.008145,580951.00


In [61]:
random_tickers = list(range(1,101))

In [62]:
#turn tickers into columns
secdata_group = secdata.set_index([secdata.groupby('Ticker #').cumcount(), 'Ticker #'])['Stock Returns'].unstack()
secdata_group

Ticker #,1,2,3,4,5,6,7,8,9,10,...,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877
0,-0.015048,-0.011842,0.030252,-0.024729,0.000246,-0.004212,-0.072416,-0.038290,0.011801,-0.004594,...,-0.012180,0.048387,0.023044,-0.018585,-0.054085,-0.005682,0.004520,0.039604,-0.007332,0.019551
1,0.026042,0.032756,0.008157,0.004754,-0.009089,0.041823,-0.072706,0.033889,0.016574,0.003750,...,-0.093711,0.030154,-0.004742,-0.002159,0.046229,-0.009000,0.012655,-0.007619,0.006250,0.048505
2,0.031472,-0.007478,0.062298,0.014196,0.017848,-0.003608,0.097044,-0.004120,0.006099,0.002299,...,0.072109,0.020311,0.021441,0.013817,0.063953,-0.004901,0.002499,0.038196,0.002259,0.054061
3,0.012795,-0.007534,0.022848,-0.005184,0.031417,0.022635,0.062683,0.009712,0.012544,-0.001433,...,-0.081218,0.004040,0.010496,-0.006897,-0.015300,0.011155,0.003601,0.053799,-0.002817,0.009441
4,0.045675,0.012042,-0.067014,0.001042,-0.026446,0.034086,0.006064,-0.004809,0.007706,-0.024117,...,0.019337,0.049624,0.047317,0.005291,0.018868,0.019341,0.026497,-0.052632,-0.000565,-0.001517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.000990,0.021250,0.009363,0.011190,0.000242,0.034339,0.019135,0.000760,0.004438,0.005749,...,0.017206,-0.003236,0.022831,-0.001548,0.005747,0.003329,-0.001074,0.002101,-0.046808,0.000000
1507,0.002308,-0.014688,-0.035250,-0.010060,-0.003867,0.012294,0.016327,0.002658,0.032401,-0.000336,...,-0.020706,0.022727,0.022321,-0.010853,0.011429,0.001161,0.005733,0.044025,-0.040179,-0.008079
1508,-0.002303,-0.004348,-0.007692,0.013720,-0.011160,-0.011861,0.012450,-0.001515,0.013442,0.009082,...,-0.011912,0.011111,-0.014556,-0.000784,-0.022599,-0.002652,-0.011756,-0.012048,0.013953,0.000000
1509,0.025387,0.016843,0.007752,-0.005013,0.006133,0.012147,0.015470,-0.002275,0.011348,0.018667,...,-0.005425,-0.003140,0.008863,0.003922,-0.011561,-0.006979,-0.019106,-0.014228,-0.050459,0.008145


In [63]:
portfolio_q2_ret = secdata_group[random_tickers]
portfolio_q2_ret

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-0.015048,-0.011842,0.030252,-0.024729,0.000246,-0.004212,-0.072416,-0.038290,0.011801,-0.004594,...,-0.010554,0.002443,0.009063,0.002205,0.006301,0.002292,0.006054,0.003861,0.003997,-0.010822
1,0.026042,0.032756,0.008157,0.004754,-0.009089,0.041823,-0.072706,0.033889,0.016574,0.003750,...,0.007267,0.035837,0.000000,0.063256,-0.051878,0.009145,-0.005731,0.003846,-0.039814,0.049599
2,0.031472,-0.007478,0.062298,0.014196,0.017848,-0.003608,0.097044,-0.004120,0.006099,0.002299,...,0.018598,0.008718,0.003208,0.002587,0.015896,-0.021296,0.007205,0.049808,0.033863,0.050035
3,0.012795,-0.007534,0.022848,-0.005184,0.031417,0.022635,0.062683,0.009712,0.012544,-0.001433,...,-0.018194,0.011524,0.013269,0.018060,0.008961,0.016620,0.001431,0.018248,0.010027,0.034414
4,0.045675,0.012042,-0.067014,0.001042,-0.026446,0.034086,0.006064,-0.004809,0.007706,-0.024117,...,0.018531,0.010443,-0.000316,0.030411,0.013806,0.018261,-0.000286,-0.020072,-0.004633,0.049264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.000990,0.021250,0.009363,0.011190,0.000242,0.034339,0.019135,0.000760,0.004438,0.005749,...,0.047939,0.001597,0.001221,0.001353,0.021475,0.013362,0.002058,0.000000,0.008584,0.007080
1507,0.002308,-0.014688,-0.035250,-0.010060,-0.003867,0.012294,0.016327,0.002658,0.032401,-0.000336,...,0.010032,-0.013335,0.004180,-0.016216,-0.009811,0.008439,-0.000790,-0.047912,0.000000,0.003047
1508,-0.002303,-0.004348,-0.007692,0.013720,-0.011160,-0.011861,0.012450,-0.001515,0.013442,0.009082,...,-0.004370,-0.008961,-0.002428,-0.010989,-0.007077,0.014121,-0.007430,0.002581,0.014894,-0.018925
1509,0.025387,0.016843,0.007752,-0.005013,0.006133,0.012147,0.015470,-0.002275,0.011348,0.018667,...,-0.012370,-0.002965,0.000348,0.008333,0.007128,0.005157,0.018474,0.000000,-0.009434,-0.014051


In [64]:
#market cap grouped by ticker #
secdata_cap_group = secdata.set_index([secdata.groupby('Ticker #').cumcount(), 
                                       'Ticker #'])['Market Capitalizations'].unstack()
secdata_cap_group

Ticker #,1,2,3,4,5,6,7,8,9,10,...,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877
0,13713091.20,32494080.25,1031397.02,237003.60,3004886.52,7.847234e+06,325464.88,6055398.00,159381.360,547023.26,...,78261.50,853027.500,300302.87,5408428.20,236966.16,16927680.00,324911.72,85800.750,974758.40,750830.04
1,14070202.95,33558466.90,1039809.72,238130.40,2977576.08,8.175431e+06,301801.76,6260608.71,162023.040,549074.40,...,70927.50,878749.560,298878.79,5396748.87,247920.80,16775330.88,329023.37,85147.030,980850.64,787249.32
2,14513021.52,33307513.95,1104587.51,241510.80,3030720.72,8.145930e+06,331089.72,6234817.20,163011.224,550336.64,...,76042.00,896597.520,305287.15,5471316.90,263776.20,16693110.72,329845.70,88399.287,983066.00,829809.06
3,14698719.63,33056561.00,1129825.61,240258.80,3125938.20,8.330311e+06,351843.44,6295371.18,165056.080,549547.74,...,69866.00,900219.606,308491.33,5433583.68,259740.28,16879315.20,331033.51,93155.100,980296.80,837643.44
4,15370089.72,33454624.30,1054111.31,240509.20,3043268.76,8.614257e+06,353977.00,6265094.19,166328.000,536294.22,...,71217.00,944892.000,323088.15,5462332.80,264641.04,17205777.60,339805.03,88252.200,979742.96,836373.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,10467246.96,15921336.52,725246.06,342293.84,3915830.78,1.882777e+08,539539.00,7588158.90,92914.360,461862.20,...,373178.07,426770.960,113816.64,1782909.00,111389.25,12836244.33,595097.02,328404.960,288061.76,580951.00
1507,10491404.80,15687485.80,699680.80,338850.24,3900689.82,1.905925e+08,548347.80,7608324.85,95924.840,461706.90,...,365451.14,436470.300,116357.19,1763559.60,112662.27,12851152.86,598508.54,342863.040,276487.85,576257.50
1508,10467246.96,15619279.34,694298.64,343499.10,3857159.56,1.883318e+08,555174.62,7596801.45,97212.450,465900.00,...,361097.94,441319.970,114663.49,1762177.50,110116.23,12817076.22,591472.28,338732.160,280345.82,576257.50
1509,10732983.20,15882361.40,699680.80,341777.30,3880817.31,1.906195e+08,563763.20,7579516.35,98315.570,474596.80,...,359139.00,439934.350,115679.71,1769088.00,108843.21,12727625.04,580171.62,333912.800,266199.93,580951.00


In [65]:
portfolio_q2_cap = secdata_cap_group[random_tickers]
portfolio_q2_cap

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,13713091.20,32494080.25,1031397.02,237003.60,3004886.52,7.847234e+06,325464.88,6055398.00,159381.360,547023.26,...,178560.000,1487771.52,80038695.00,3172555.44,2102399.000,724662.450,282061.80,2071511.00,476468.19,373062.81
1,14070202.95,33558466.90,1039809.72,238130.40,2977576.08,8.175431e+06,301801.76,6260608.71,162023.040,549074.40,...,179857.536,1541089.02,80038695.00,3373239.64,1993330.000,731289.450,280445.40,2079478.35,457497.99,391566.29
2,14513021.52,33307513.95,1104587.51,241510.80,3030720.72,8.145930e+06,331089.72,6234817.20,163011.224,550336.64,...,183202.560,1554525.03,80295435.00,3381965.04,2025016.425,715716.000,282465.90,2183053.90,472990.32,411158.21
3,14698719.63,33056561.00,1129825.61,240258.80,3125938.20,8.330311e+06,351843.44,6295371.18,165056.080,549547.74,...,179869.440,1572439.71,81360906.00,3443042.84,2043163.250,727611.465,282870.00,2222890.65,477732.87,425307.93
4,15370089.72,33454624.30,1054111.31,240509.20,3043268.76,8.614257e+06,353977.00,6265094.19,166328.000,536294.22,...,183202.560,1588861.50,81335232.00,3547747.64,2071370.750,740898.600,282789.18,2178273.49,475519.68,446260.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,10467246.96,15921336.52,725246.06,342293.84,3915830.78,1.882777e+08,539539.00,7588158.90,92914.360,461862.20,...,1393725.760,2901305.46,58116963.96,1355043.60,2022672.610,1411439.280,583401.65,2706191.84,297453.60,1887976.82
1507,10491404.80,15687485.80,699680.80,338850.24,3900689.82,1.905925e+08,548347.80,7608324.85,95924.840,461706.90,...,1407707.760,2862615.78,58359877.08,1333069.92,2002828.590,1423350.160,582940.90,2576534.00,297453.60,1893728.80
1508,10467246.96,15619279.34,694298.64,343499.10,3857159.56,1.883318e+08,555174.62,7596801.45,97212.450,465900.00,...,1401555.680,2836962.84,58218177.76,1318420.80,1988654.290,1443449.770,578609.85,2583183.12,301883.76,1857889.54
1509,10732983.20,15882361.40,699680.80,341777.30,3880817.31,1.906195e+08,563763.20,7579516.35,98315.570,474596.80,...,1384218.000,2828552.04,58238420.52,1329407.64,2002828.590,1450894.070,589299.25,2583183.12,299035.80,1831784.40


For part b:

Standard deviation of portfolio = portfolio volatility.

Equation:
$$ \hat{\sigma}_{Portfolio} = \sqrt{w_{T} \cdot \Sigma \cdot w}$$

where:
- $w$ is portfolio weights
- $\Sigma$ is covariance matrix
- $\cdot$ the dot-multiplication for matrix multiplication
- $\hat{\sigma}_{Portfolio}$ is the estimated portfolio volatility/standard deviation

In [66]:
#function to find portfolio standard deviation
def sd_portfolio(cov_mat, arr_weights):
    if np.isnan(cov_mat).any():
        return cov_mat
    return np.dot(np.dot(np.transpose(arr_weights), cov_mat), arr_weights)**0.5

### Rolling Window 504

#### i) Generate a covariance matrixes for generated portfolio.

In [67]:
cov_df_504 = portfolio_q2_ret.rolling(504).cov()
cov_df_504.dropna(inplace = True)
cov_df_504.drop(cov_df_504.tail(100).index, inplace = True)
cov_df_504_np = cov_df_504.to_numpy()
cov_df_504

Ticker #            1         2         3         4         5         6    \
     Ticker #                                                               
503  1         0.000438  0.000085  0.000203  0.000117  0.000071  0.000140   
     2         0.000085  0.000255  0.000123  0.000068  0.000062  0.000122   
     3         0.000203  0.000123  0.000910  0.000135  0.000131  0.000171   
     4         0.000117  0.000068  0.000135  0.000482  0.000065  0.000090   
     5         0.000071  0.000062  0.000131  0.000065  0.000316  0.000070   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000409  0.000568  0.000574  0.000460  0.000312  0.000247   
     97        0.000689  0.001155  0.001032  0.000897  0.000628  0.000641   
     98        0.000728  0.000893  0.002917  0.001070  0.000794  0.000664   
     99        0.000591  0.000803  0.000918  0.000784  0.000431  0.000454   
     100       0.000938  0.001578  0.001011  0.000973  0.000742  0.000784   

Ticker #            7         8         9         10   ...       91   \
     Ticker #                                          ...             
503  1         0.000142  0.000058  0.000109  0.000071  ...  0.000090   
     2         0.000086  0.000053  0.000103  0.000056  ...  0.000067   
     3         0.000280  0.000100  0.000219  0.000118  ...  0.000065   
     4         0.000122  0.000040  0.000112  0.000053  ...  0.000098   
     5         0.000099  0.000007  0.000089  0.000046  ...  0.000053   
...                 ...       ...       ...       ...  ...       ...   
1509 96        0.000448  0.000218  0.000859  0.000369  ...  0.000440   
     97        0.000685  0.000323  0.001328  0.000542  ...  0.000650   
     98        0.000785  0.000371  0.002209  0.000610  ...  0.000686   
     99        0.000610  0.000323  0.001325  0.000373  ...  0.000555   
     100       0.000960  0.000419  0.001536  0.000670  ...  0.000831   

Ticker #            92        93        94        95        96        97   \
     Ticker #                                                               
503  1         0.000099  0.000073  0.000305  0.000144  0.000132  0.000099   
     2         0.000076  0.000060  0.000163  0.000070  0.000094  0.000068   
     3         0.000131  0.000102  0.000325  0.000197  0.000201  0.000158   
     4         0.000061  0.000055  0.000150  0.000129  0.000100  0.000068   
     5         0.000055  0.000026  0.000087  0.000073  0.000069  0.000052   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000635  0.000191  0.000517  0.000474  0.000999  0.000463   
     97        0.001374  0.000342  0.001073  0.000813  0.000463  0.002264   
     98        0.001612  0.000385  0.001333  0.001108  0.000618  0.001147   
     99        0.000910  0.000320  0.000834  0.000789  0.000528  0.000799   
     100       0.001517  0.000393  0.001317  0.000982  0.000582  0.001299   

Ticker #            98        99        100  
     Ticker #                                
503  1         0.000189  0.000152  0.000228  
     2         0.000128  0.000106  0.000155  
     3         0.000648  0.000197  0.000246  
     4         0.000107  0.000150  0.000127  
     5         0.000157  0.000082  0.000101  
...                 ...       ...       ...  
1509 96        0.000618  0.000528  0.000582  
     97        0.001147  0.000799  0.001299  
     98        0.004659  0.000917  0.001250  
     99        0.000917  0.001717  0.001058  
     100       0.001250  0.001058  0.003372  

[100700 rows x 100 columns]

#### ii) Estimate the standard deviations of the portfolio over rw (from the last day in the rolling window).


In [68]:
#portfolio_weights
weights_q2_504 = portfolio_q2_cap.iloc[:, :].apply(lambda x: x.div(x.sum()), axis=1)
df_weights_q2_504 = weights_q2_504.drop(weights_q2_504.tail(1).index)
arr_weights_q2_504 = df_weights_q2_504.tail(1007).to_numpy()
df_weights_q2_504

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.018568,0.043998,0.001397,0.000321,0.004069,0.010625,0.000441,0.008199,0.000216,0.000741,...,0.000242,0.002014,0.108375,0.004296,0.002847,0.000981,0.000382,0.002805,0.000645,0.000505
1,0.018809,0.044861,0.001390,0.000318,0.003980,0.010929,0.000403,0.008369,0.000217,0.000734,...,0.000240,0.002060,0.106997,0.004509,0.002665,0.000978,0.000375,0.002780,0.000612,0.000523
2,0.019375,0.044465,0.001475,0.000322,0.004046,0.010875,0.000442,0.008323,0.000218,0.000735,...,0.000245,0.002075,0.107192,0.004515,0.002703,0.000955,0.000377,0.002914,0.000631,0.000549
3,0.019478,0.043805,0.001497,0.000318,0.004142,0.011039,0.000466,0.008342,0.000219,0.000728,...,0.000238,0.002084,0.107815,0.004563,0.002707,0.000964,0.000375,0.002946,0.000633,0.000564
4,0.020277,0.044135,0.001391,0.000317,0.004015,0.011364,0.000467,0.008265,0.000219,0.000708,...,0.000242,0.002096,0.107301,0.004680,0.002733,0.000977,0.000373,0.002874,0.000627,0.000589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [69]:
sd_port_504 = []
for i in range(0, len(cov_df_504_np), 100):
    idx = 0
    sd_port_504.append(sd_portfolio(cov_df_504_np[i:i+100], arr_weights_q2_504[idx]))
    idx += 1
sd_port_504_arr = np.array(sd_port_504)
sd_port_504_arr

array([0.00820984, 0.00824556, 0.00823351, ..., 0.03161844, 0.03161853,
       0.03161824])

#### iii) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [70]:
#getting one-day ahead returns array
dayahead504_port_ret_q2 = []
dayahead504_ret_q2 = portfolio_q2_ret.loc[504:1510].to_numpy()
dayahead504_w_q2 = weights_q2_504.loc[504:1510].to_numpy()
for i in range(0, len(dayahead504_w_q2)):
    dayahead504_port_ret_q2.append(np.multiply(dayahead504_ret_q2[i], dayahead504_w_q2[i]))
dayahead504_port_ret_q2_arr = np.sum(np.array(dayahead504_port_ret_q2), axis = 1)
dayahead504_port_ret_q2_arr

array([ 0.01826034,  0.00564215, -0.00055066, ..., -0.00414611,
        0.00514476, -0.00946444])

#### iv) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [71]:
#dividing arrays to get standardized outcomes.
standardized_outcomes_504_q2 = dayahead504_port_ret_q2_arr / sd_port_504_arr
std_outcomes_504_q2 = pd.DataFrame(standardized_outcomes_504_q2)
std_outcomes_504_q2.index += 504
std_outcomes_504_q2.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_504_q2

,Standardized Outcome
504,2.224203
505,0.684265
506,-0.066881
507,1.336911
508,0.451453
...,...
1506,0.382310
1507,0.164140
1508,-0.131129
1509,0.162713


### Rolling Window 252

#### i) Generate a covariance matrixes for generated portfolio.

In [72]:
cov_df_252 = portfolio_q2_ret.rolling(252).cov()
cov_df_252.dropna(inplace = True)
cov_df_252.drop(cov_df_252.tail(100).index, inplace = True)
cov_df_252_np = cov_df_252.to_numpy()
cov_df_252

Ticker #            1         2         3         4         5         6    \
     Ticker #                                                               
251  1         0.000583  0.000118  0.000287  0.000161  0.000108  0.000154   
     2         0.000118  0.000327  0.000146  0.000086  0.000071  0.000127   
     3         0.000287  0.000146  0.000980  0.000183  0.000155  0.000187   
     4         0.000161  0.000086  0.000183  0.000538  0.000077  0.000067   
     5         0.000108  0.000071  0.000155  0.000077  0.000305  0.000057   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000361  0.000421  0.000348  0.000372  0.000114  0.000126   
     97        0.000467  0.000730  0.000631  0.000512  0.000308  0.000345   
     98        0.000628  0.001012  0.001642  0.000609  0.000306  0.000381   
     99        0.000455  0.000677  0.000675  0.000526  0.000225  0.000251   
     100       0.000725  0.001194  0.000937  0.000610  0.000317  0.000482   

Ticker #            7         8         9         10   ...       91   \
     Ticker #                                          ...             
251  1         0.000203  0.000078  0.000128  0.000072  ...  0.000105   
     2         0.000116  0.000072  0.000120  0.000072  ...  0.000063   
     3         0.000392  0.000126  0.000202  0.000104  ...  0.000034   
     4         0.000177  0.000052  0.000127  0.000032  ...  0.000162   
     5         0.000108  0.000014  0.000094  0.000044  ...  0.000076   
...                 ...       ...       ...       ...  ...       ...   
1509 96        0.000305  0.000147  0.000851  0.000360  ...  0.000367   
     97        0.000420  0.000180  0.001077  0.000350  ...  0.000469   
     98        0.000467  0.000249  0.001951  0.000427  ...  0.000439   
     99        0.000378  0.000226  0.001227  0.000412  ...  0.000414   
     100       0.000615  0.000217  0.001320  0.000475  ...  0.000546   

Ticker #            92        93        94        95        96        97   \
     Ticker #                                                               
251  1         0.000145  0.000087  0.000411  0.000210  0.000177  0.000116   
     2         0.000095  0.000071  0.000164  0.000109  0.000085  0.000071   
     3         0.000155  0.000115  0.000305  0.000284  0.000200  0.000122   
     4         0.000074  0.000063  0.000237  0.000184  0.000103  0.000053   
     5         0.000059  0.000038  0.000078  0.000112  0.000063  0.000044   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000507  0.000108  0.000353  0.000247  0.001040  0.000295   
     97        0.000872  0.000166  0.000786  0.000386  0.000295  0.001199   
     98        0.001183  0.000304  0.001120  0.000602  0.000343  0.000654   
     99        0.000791  0.000198  0.000722  0.000447  0.000368  0.000468   
     100       0.001258  0.000212  0.001219  0.000598  0.000332  0.000781   

Ticker #            98        99        100  
     Ticker #                                
251  1         0.000282  0.000224  0.000307  
     2         0.000163  0.000131  0.000200  
     3         0.000722  0.000247  0.000230  
     4         0.000131  0.000237  0.000161  
     5         0.000170  0.000116  0.000122  
...                 ...       ...       ...  
1509 96        0.000343  0.000368  0.000332  
     97        0.000654  0.000468  0.000781  
     98        0.003277  0.000616  0.001037  
     99        0.000616  0.001219  0.000772  
     100       0.001037  0.000772  0.002278  

[125900 rows x 100 columns]

#### ii) Estimate the standard deviations of the portfolio over rw (from the last day in the rolling window).

In [73]:
#portfolio_weights
weights_q2_252 = portfolio_q2_cap.iloc[:, :].apply(lambda x: x.div(x.sum()), axis=1)
df_weights_q2_252 = weights_q2_252.drop(weights_q2_252.tail(1).index)
arr_weights_q2_252 = df_weights_q2_252.tail(1259).to_numpy()
df_weights_q2_252

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.018568,0.043998,0.001397,0.000321,0.004069,0.010625,0.000441,0.008199,0.000216,0.000741,...,0.000242,0.002014,0.108375,0.004296,0.002847,0.000981,0.000382,0.002805,0.000645,0.000505
1,0.018809,0.044861,0.001390,0.000318,0.003980,0.010929,0.000403,0.008369,0.000217,0.000734,...,0.000240,0.002060,0.106997,0.004509,0.002665,0.000978,0.000375,0.002780,0.000612,0.000523
2,0.019375,0.044465,0.001475,0.000322,0.004046,0.010875,0.000442,0.008323,0.000218,0.000735,...,0.000245,0.002075,0.107192,0.004515,0.002703,0.000955,0.000377,0.002914,0.000631,0.000549
3,0.019478,0.043805,0.001497,0.000318,0.004142,0.011039,0.000466,0.008342,0.000219,0.000728,...,0.000238,0.002084,0.107815,0.004563,0.002707,0.000964,0.000375,0.002946,0.000633,0.000564
4,0.020277,0.044135,0.001391,0.000317,0.004015,0.011364,0.000467,0.008265,0.000219,0.000708,...,0.000242,0.002096,0.107301,0.004680,0.002733,0.000977,0.000373,0.002874,0.000627,0.000589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [74]:
sd_port_252 = []
for i in range(0, len(cov_df_252_np), 100):
    idx = 0
    sd_port_252.append(sd_portfolio(cov_df_252_np[i:i+100], arr_weights_q2_252[idx]))
    idx += 1
sd_port_252_arr = np.array(sd_port_252)
sd_port_252_arr

array([0.00848636, 0.00849811, 0.00850478, ..., 0.03044609, 0.03043997,
       0.03041131])

#### iii) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [75]:
#getting one-day ahead returns array
dayahead252_port_ret_q2 = []
dayahead252_ret_q2 = portfolio_q2_ret.loc[252:1510].to_numpy()
dayahead252_w_q2 = weights_q2_252.loc[252:1510].to_numpy()
for i in range(0, len(dayahead252_w_q2)):
    dayahead252_port_ret_q2.append(np.multiply(dayahead252_ret_q2[i], dayahead252_w_q2[i]))
dayahead252_port_ret_q2_arr = np.sum(np.array(dayahead252_port_ret_q2), axis = 1)
dayahead252_port_ret_q2_arr

array([-0.00643605, -0.01146078, -0.00250408, ..., -0.00414611,
        0.00514476, -0.00946444])

#### iv) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [76]:
standardized_outcomes_252_q2 = dayahead252_port_ret_q2_arr / sd_port_252_arr
std_outcomes_252_q2 = pd.DataFrame(standardized_outcomes_252_q2)
std_outcomes_252_q2.index += 252
std_outcomes_252_q2.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_252_q2

,Standardized Outcome
252,-0.758400
253,-1.348626
254,-0.294432
255,0.541728
256,0.640736
...,...
1506,0.397123
1507,0.170497
1508,-0.136179
1509,0.169013


### Rolling Window 126

#### i) Generate a covariance matrixes for generated portfolio.

In [77]:
cov_df_126 = portfolio_q2_ret.rolling(126).cov()
cov_df_126.dropna(inplace = True)
cov_df_126.drop(cov_df_126.tail(100).index, inplace = True)
cov_df_126_np = cov_df_126.to_numpy()
cov_df_126

Ticker #            1         2         3         4         5         6    \
     Ticker #                                                               
125  1         0.000528  0.000192  0.000244  0.000163  0.000079  0.000183   
     2         0.000192  0.000408  0.000228  0.000095  0.000062  0.000162   
     3         0.000244  0.000228  0.001045  0.000181  0.000174  0.000204   
     4         0.000163  0.000095  0.000181  0.000595  0.000054  0.000046   
     5         0.000079  0.000062  0.000174  0.000054  0.000228  0.000089   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000177  0.000201  0.000162  0.000103  0.000020  0.000089   
     97        0.000233  0.000360  0.000289  0.000181  0.000154  0.000157   
     98        0.000241  0.000462  0.000875  0.000252  0.000170  0.000176   
     99        0.000180  0.000191  0.000192  0.000191  0.000055  0.000075   
     100       0.000318  0.000584  0.000173  0.000194  0.000197  0.000283   

Ticker #            7         8         9         10   ...       91   \
     Ticker #                                          ...             
125  1         0.000176  0.000096  0.000134  0.000074  ...  0.000072   
     2         0.000170  0.000098  0.000113  0.000089  ... -0.000003   
     3         0.000361  0.000105  0.000182  0.000082  ... -0.000171   
     4         0.000164  0.000057  0.000104 -0.000036  ...  0.000159   
     5         0.000104  0.000034  0.000074  0.000025  ... -0.000010   
...                 ...       ...       ...       ...  ...       ...   
1509 96        0.000153  0.000083  0.000255  0.000073  ...  0.000204   
     97        0.000230  0.000081  0.000356  0.000235  ...  0.000274   
     98        0.000315  0.000113  0.000606  0.000178  ... -0.000012   
     99        0.000078  0.000078  0.000347  0.000153  ...  0.000181   
     100       0.000283  0.000072  0.000349  0.000379  ...  0.000278   

Ticker #            92        93        94        95        96        97   \
     Ticker #                                                               
125  1         0.000158  0.000109  0.000395  0.000195  0.000190  0.000121   
     2         0.000153  0.000061  0.000269  0.000127  0.000083  0.000092   
     3         0.000142  0.000102  0.000360  0.000273  0.000196  0.000071   
     4         0.000060  0.000032  0.000350  0.000156  0.000092  0.000008   
     5         0.000060  0.000022  0.000081  0.000106  0.000066  0.000014   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000131  0.000038  0.000215  0.000115  0.000490  0.000195   
     97        0.000307  0.000040  0.000395  0.000139  0.000195  0.000858   
     98        0.000405  0.000141  0.000510  0.000322  0.000198  0.000437   
     99        0.000193  0.000039  0.000235  0.000129  0.000204  0.000252   
     100       0.000596  0.000039  0.000630  0.000213  0.000103  0.000528   

Ticker #            98        99        100  
     Ticker #                                
125  1         0.000300  0.000233  0.000311  
     2         0.000225  0.000130  0.000278  
     3         0.000716  0.000275  0.000173  
     4         0.000111  0.000233  0.000097  
     5         0.000199  0.000079  0.000131  
...                 ...       ...       ...  
1509 96        0.000198  0.000204  0.000103  
     97        0.000437  0.000252  0.000528  
     98        0.001935  0.000186  0.000519  
     99        0.000186  0.000673  0.000261  
     100       0.000519  0.000261  0.001785  

[138500 rows x 100 columns]

#### ii) Estimate the standard deviations of the portfolio over rw (from the last day in the rolling window).

In [78]:
#portfolio_weights
weights_q2_126 = portfolio_q2_cap.iloc[:, :].apply(lambda x: x.div(x.sum()), axis=1)
df_weights_q2_126 = weights_q2_126.drop(weights_q2_126.tail(1).index)
arr_weights_q2_126 = df_weights_q2_126.tail(1385).to_numpy()
df_weights_q2_126

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.018568,0.043998,0.001397,0.000321,0.004069,0.010625,0.000441,0.008199,0.000216,0.000741,...,0.000242,0.002014,0.108375,0.004296,0.002847,0.000981,0.000382,0.002805,0.000645,0.000505
1,0.018809,0.044861,0.001390,0.000318,0.003980,0.010929,0.000403,0.008369,0.000217,0.000734,...,0.000240,0.002060,0.106997,0.004509,0.002665,0.000978,0.000375,0.002780,0.000612,0.000523
2,0.019375,0.044465,0.001475,0.000322,0.004046,0.010875,0.000442,0.008323,0.000218,0.000735,...,0.000245,0.002075,0.107192,0.004515,0.002703,0.000955,0.000377,0.002914,0.000631,0.000549
3,0.019478,0.043805,0.001497,0.000318,0.004142,0.011039,0.000466,0.008342,0.000219,0.000728,...,0.000238,0.002084,0.107815,0.004563,0.002707,0.000964,0.000375,0.002946,0.000633,0.000564
4,0.020277,0.044135,0.001391,0.000317,0.004015,0.011364,0.000467,0.008265,0.000219,0.000708,...,0.000242,0.002096,0.107301,0.004680,0.002733,0.000977,0.000373,0.002874,0.000627,0.000589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [79]:
sd_port_126 = []
for i in range(0, len(cov_df_126_np), 100):
    idx = 0
    sd_port_126.append(sd_portfolio(cov_df_126_np[i:i+100], arr_weights_q2_126[idx]))
    idx += 1
sd_port_126_arr = np.array(sd_port_126)
sd_port_126_arr

array([0.00840445, 0.0084857 , 0.00841868, ..., 0.02738999, 0.02715904,
       0.02676144])

#### iii) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [80]:
#getting one-day ahead returns array
dayahead126_port_ret_q2 = []
dayahead126_ret_q2 = portfolio_q2_ret.loc[126:1510].to_numpy()
dayahead126_w_q2 = weights_q2_126.loc[126:1510].to_numpy()
for i in range(0, len(dayahead126_w_q2)):
    dayahead126_port_ret_q2.append(np.multiply(dayahead126_ret_q2[i], dayahead126_w_q2[i]))
dayahead126_port_ret_q2_arr = np.sum(np.array(dayahead126_port_ret_q2), axis = 1)
dayahead126_port_ret_q2_arr

array([-0.01262911,  0.00056176, -0.0061875 , ..., -0.00414611,
        0.00514476, -0.00946444])

#### iv) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [81]:
standardized_outcomes_126_q2 = dayahead126_port_ret_q2_arr / sd_port_126_arr
std_outcomes_126_q2 = pd.DataFrame(standardized_outcomes_126_q2)
std_outcomes_126_q2.index += 126
std_outcomes_126_q2.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_126_q2

,Standardized Outcome
126,-1.502669
127,0.066201
128,-0.734973
129,-0.016849
130,0.009584
...,...
1506,0.440712
1507,0.189176
1508,-0.151373
1509,0.189431


### Rolling Window 63

#### i) Generate a covariance matrixes for generated portfolio.

In [82]:
cov_df_63 = portfolio_q2_ret.rolling(63).cov()
cov_df_63.dropna(inplace = True)
cov_df_63.drop(cov_df_63.tail(100).index, inplace = True)
cov_df_63_np = cov_df_63.to_numpy()
cov_df_63

Ticker #            1         2         3         4         5         6    \
     Ticker #                                                               
62   1         0.000632  0.000198  0.000263  0.000097  0.000066  0.000207   
     2         0.000198  0.000391  0.000206 -0.000027  0.000041  0.000140   
     3         0.000263  0.000206  0.001327  0.000140  0.000238  0.000196   
     4         0.000097 -0.000027  0.000140  0.000626  0.000007 -0.000044   
     5         0.000066  0.000041  0.000238  0.000007  0.000237  0.000092   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000241  0.000316  0.000219  0.000169  0.000143  0.000141   
     97        0.000248  0.000427  0.000427  0.000238  0.000290  0.000161   
     98        0.000271  0.000581  0.001058  0.000278  0.000306  0.000132   
     99        0.000113  0.000208  0.000064  0.000106  0.000098  0.000015   
     100       0.000247  0.000608  0.000099  0.000176  0.000212  0.000252   

Ticker #            7         8         9         10   ...       91   \
     Ticker #                                          ...             
62   1         0.000192  0.000119  0.000127  0.000069  ...  0.000044   
     2         0.000150  0.000113  0.000087  0.000063  ... -0.000027   
     3         0.000387  0.000139  0.000236  0.000080  ... -0.000193   
     4         0.000040  0.000023  0.000009 -0.000157  ...  0.000173   
     5         0.000085  0.000040  0.000074  0.000034  ...  0.000007   
...                 ...       ...       ...       ...  ...       ...   
1509 96        0.000228  0.000099  0.000243  0.000192  ...  0.000170   
     97        0.000354  0.000147  0.000544  0.000295  ...  0.000281   
     98        0.000350  0.000154  0.000898  0.000320  ...  0.000180   
     99        0.000145  0.000074  0.000201  0.000155  ...  0.000202   
     100       0.000140  0.000189  0.000425  0.000261  ...  0.000403   

Ticker #            92        93        94        95        96        97   \
     Ticker #                                                               
62   1         0.000166  0.000138  0.000372  0.000155  0.000232  0.000152   
     2         0.000124  0.000045  0.000211  0.000142  0.000049  0.000108   
     3         0.000124  0.000120  0.000358  0.000273  0.000251  0.000155   
     4        -0.000012  0.000003  0.000261  0.000161  0.000085 -0.000010   
     5         0.000037  0.000053  0.000157  0.000060  0.000080  0.000018   
...                 ...       ...       ...       ...       ...       ...   
1509 96        0.000259  0.000157  0.000408  0.000167  0.000509  0.000317   
     97        0.000304  0.000077  0.000583  0.000215  0.000317  0.000901   
     98        0.000483  0.000188  0.000864  0.000380  0.000435  0.000627   
     99        0.000144  0.000059  0.000261  0.000141  0.000153  0.000297   
     100       0.000498  0.000174  0.000415  0.000194  0.000350  0.000506   

Ticker #            98        99        100  
     Ticker #                                
62   1         0.000395  0.000191  0.000463  
     2         0.000249  0.000079  0.000386  
     3         0.000948  0.000307  0.000156  
     4        -0.000030  0.000118  0.000128  
     5         0.000184  0.000053  0.000137  
...                 ...       ...       ...  
1509 96        0.000435  0.000153  0.000350  
     97        0.000627  0.000297  0.000506  
     98        0.001634  0.000195  0.000462  
     99        0.000195  0.000371  0.000177  
     100       0.000462  0.000177  0.001499  

[144800 rows x 100 columns]

#### ii) Estimate the standard deviations of the portfolio over rw (from the last day in the rolling window).

In [83]:
#portfolio_weights
weights_q2_63 = portfolio_q2_cap.iloc[:, :].apply(lambda x: x.div(x.sum()), axis=1)
df_weights_q2_63 = weights_q2_63.drop(weights_q2_63.tail(1).index)
arr_weights_q2_63 = df_weights_q2_63.tail(1448).to_numpy()
df_weights_q2_63

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.018568,0.043998,0.001397,0.000321,0.004069,0.010625,0.000441,0.008199,0.000216,0.000741,...,0.000242,0.002014,0.108375,0.004296,0.002847,0.000981,0.000382,0.002805,0.000645,0.000505
1,0.018809,0.044861,0.001390,0.000318,0.003980,0.010929,0.000403,0.008369,0.000217,0.000734,...,0.000240,0.002060,0.106997,0.004509,0.002665,0.000978,0.000375,0.002780,0.000612,0.000523
2,0.019375,0.044465,0.001475,0.000322,0.004046,0.010875,0.000442,0.008323,0.000218,0.000735,...,0.000245,0.002075,0.107192,0.004515,0.002703,0.000955,0.000377,0.002914,0.000631,0.000549
3,0.019478,0.043805,0.001497,0.000318,0.004142,0.011039,0.000466,0.008342,0.000219,0.000728,...,0.000238,0.002084,0.107815,0.004563,0.002707,0.000964,0.000375,0.002946,0.000633,0.000564
4,0.020277,0.044135,0.001391,0.000317,0.004015,0.011364,0.000467,0.008265,0.000219,0.000708,...,0.000242,0.002096,0.107301,0.004680,0.002733,0.000977,0.000373,0.002874,0.000627,0.000589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [84]:
sd_port_63 = []
for i in range(0, len(cov_df_63_np), 100):
    idx = 0
    sd_port_63.append(sd_portfolio(cov_df_63_np[i:i+100], arr_weights_q2_63[idx]))
    idx += 1
sd_port_63_arr = np.array(sd_port_63)
sd_port_63_arr

array([0.00885192, 0.00908816, 0.00906737, ..., 0.01689205, 0.01687095,
       0.01685115])

#### iii) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [85]:
#getting one-day ahead returns array
dayahead63_port_ret_q2 = []
dayahead63_ret_q2 = portfolio_q2_ret.loc[63:1510].to_numpy()
dayahead63_w_q2 = weights_q2_63.loc[63:1510].to_numpy()
for i in range(0, len(dayahead63_w_q2)):
    dayahead63_port_ret_q2.append(np.multiply(dayahead63_ret_q2[i], dayahead63_w_q2[i]))
dayahead63_port_ret_q2_arr = np.sum(np.array(dayahead63_port_ret_q2), axis = 1)
dayahead63_port_ret_q2_arr

array([ 0.01740121,  0.0116785 , -0.00496981, ..., -0.00414611,
        0.00514476, -0.00946444])

#### iv) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [86]:
standardized_outcomes_63_q2 = dayahead63_port_ret_q2_arr / sd_port_63_arr
std_outcomes_63_q2 = pd.DataFrame(standardized_outcomes_63_q2)
std_outcomes_63_q2.index += 63
std_outcomes_63_q2.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_63_q2

,Standardized Outcome
63,1.965812
64,1.285023
65,-0.548099
66,-0.525563
67,0.197667
...,...
1506,0.706105
1507,0.302347
1508,-0.245447
1509,0.304948


## b) Compute bias statistics.

In [87]:
bias_stat_q2_504 = np.std(standardized_outcomes_504_q2)
bias_stat_q2_252 = np.std(standardized_outcomes_252_q2)
bias_stat_q2_126 = np.std(standardized_outcomes_126_q2)
bias_stat_q2_63 = np.std(standardized_outcomes_63_q2)
print(bias_stat_q2_504, bias_stat_q2_252, bias_stat_q2_126, bias_stat_q2_63)

1.1687924023168035 1.0366178517316669 0.9687974333689725 0.946844788289987


Rolling window 63 gives closest bias statistic to 1. Idk why.

# Question 3

We now consider the market model approach to estimating volatility.  Each step below shouldbe completed using 504, 252, 126, and 63 day rolling windows.

Portfolio from part 2:

In [88]:
portfolio_q2_ret

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-0.015048,-0.011842,0.030252,-0.024729,0.000246,-0.004212,-0.072416,-0.038290,0.011801,-0.004594,...,-0.010554,0.002443,0.009063,0.002205,0.006301,0.002292,0.006054,0.003861,0.003997,-0.010822
1,0.026042,0.032756,0.008157,0.004754,-0.009089,0.041823,-0.072706,0.033889,0.016574,0.003750,...,0.007267,0.035837,0.000000,0.063256,-0.051878,0.009145,-0.005731,0.003846,-0.039814,0.049599
2,0.031472,-0.007478,0.062298,0.014196,0.017848,-0.003608,0.097044,-0.004120,0.006099,0.002299,...,0.018598,0.008718,0.003208,0.002587,0.015896,-0.021296,0.007205,0.049808,0.033863,0.050035
3,0.012795,-0.007534,0.022848,-0.005184,0.031417,0.022635,0.062683,0.009712,0.012544,-0.001433,...,-0.018194,0.011524,0.013269,0.018060,0.008961,0.016620,0.001431,0.018248,0.010027,0.034414
4,0.045675,0.012042,-0.067014,0.001042,-0.026446,0.034086,0.006064,-0.004809,0.007706,-0.024117,...,0.018531,0.010443,-0.000316,0.030411,0.013806,0.018261,-0.000286,-0.020072,-0.004633,0.049264
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.000990,0.021250,0.009363,0.011190,0.000242,0.034339,0.019135,0.000760,0.004438,0.005749,...,0.047939,0.001597,0.001221,0.001353,0.021475,0.013362,0.002058,0.000000,0.008584,0.007080
1507,0.002308,-0.014688,-0.035250,-0.010060,-0.003867,0.012294,0.016327,0.002658,0.032401,-0.000336,...,0.010032,-0.013335,0.004180,-0.016216,-0.009811,0.008439,-0.000790,-0.047912,0.000000,0.003047
1508,-0.002303,-0.004348,-0.007692,0.013720,-0.011160,-0.011861,0.012450,-0.001515,0.013442,0.009082,...,-0.004370,-0.008961,-0.002428,-0.010989,-0.007077,0.014121,-0.007430,0.002581,0.014894,-0.018925
1509,0.025387,0.016843,0.007752,-0.005013,0.006133,0.012147,0.015470,-0.002275,0.011348,0.018667,...,-0.012370,-0.002965,0.000348,0.008333,0.007128,0.005157,0.018474,0.000000,-0.009434,-0.014051


In [89]:
portfolio_q2_cap

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,13713091.20,32494080.25,1031397.02,237003.60,3004886.52,7.847234e+06,325464.88,6055398.00,159381.360,547023.26,...,178560.000,1487771.52,80038695.00,3172555.44,2102399.000,724662.450,282061.80,2071511.00,476468.19,373062.81
1,14070202.95,33558466.90,1039809.72,238130.40,2977576.08,8.175431e+06,301801.76,6260608.71,162023.040,549074.40,...,179857.536,1541089.02,80038695.00,3373239.64,1993330.000,731289.450,280445.40,2079478.35,457497.99,391566.29
2,14513021.52,33307513.95,1104587.51,241510.80,3030720.72,8.145930e+06,331089.72,6234817.20,163011.224,550336.64,...,183202.560,1554525.03,80295435.00,3381965.04,2025016.425,715716.000,282465.90,2183053.90,472990.32,411158.21
3,14698719.63,33056561.00,1129825.61,240258.80,3125938.20,8.330311e+06,351843.44,6295371.18,165056.080,549547.74,...,179869.440,1572439.71,81360906.00,3443042.84,2043163.250,727611.465,282870.00,2222890.65,477732.87,425307.93
4,15370089.72,33454624.30,1054111.31,240509.20,3043268.76,8.614257e+06,353977.00,6265094.19,166328.000,536294.22,...,183202.560,1588861.50,81335232.00,3547747.64,2071370.750,740898.600,282789.18,2178273.49,475519.68,446260.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,10467246.96,15921336.52,725246.06,342293.84,3915830.78,1.882777e+08,539539.00,7588158.90,92914.360,461862.20,...,1393725.760,2901305.46,58116963.96,1355043.60,2022672.610,1411439.280,583401.65,2706191.84,297453.60,1887976.82
1507,10491404.80,15687485.80,699680.80,338850.24,3900689.82,1.905925e+08,548347.80,7608324.85,95924.840,461706.90,...,1407707.760,2862615.78,58359877.08,1333069.92,2002828.590,1423350.160,582940.90,2576534.00,297453.60,1893728.80
1508,10467246.96,15619279.34,694298.64,343499.10,3857159.56,1.883318e+08,555174.62,7596801.45,97212.450,465900.00,...,1401555.680,2836962.84,58218177.76,1318420.80,1988654.290,1443449.770,578609.85,2583183.12,301883.76,1857889.54
1509,10732983.20,15882361.40,699680.80,341777.30,3880817.31,1.906195e+08,563763.20,7579516.35,98315.570,474596.80,...,1384218.000,2828552.04,58238420.52,1329407.64,2002828.590,1450894.070,589299.25,2583183.12,299035.80,1831784.40


In [90]:
df_weights = df_weights_q2_504
df_weights

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.018568,0.043998,0.001397,0.000321,0.004069,0.010625,0.000441,0.008199,0.000216,0.000741,...,0.000242,0.002014,0.108375,0.004296,0.002847,0.000981,0.000382,0.002805,0.000645,0.000505
1,0.018809,0.044861,0.001390,0.000318,0.003980,0.010929,0.000403,0.008369,0.000217,0.000734,...,0.000240,0.002060,0.106997,0.004509,0.002665,0.000978,0.000375,0.002780,0.000612,0.000523
2,0.019375,0.044465,0.001475,0.000322,0.004046,0.010875,0.000442,0.008323,0.000218,0.000735,...,0.000245,0.002075,0.107192,0.004515,0.002703,0.000955,0.000377,0.002914,0.000631,0.000549
3,0.019478,0.043805,0.001497,0.000318,0.004142,0.011039,0.000466,0.008342,0.000219,0.000728,...,0.000238,0.002084,0.107815,0.004563,0.002707,0.000964,0.000375,0.002946,0.000633,0.000564
4,0.020277,0.044135,0.001391,0.000317,0.004015,0.011364,0.000467,0.008265,0.000219,0.000708,...,0.000242,0.002096,0.107301,0.004680,0.002733,0.000977,0.000373,0.002874,0.000627,0.000589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [20]:
#get risk premium df
eqt_risk_prem_df = portfolio_q2_ret.sub(ffdata['Risk-free rate'], axis=0)
eqt_risk_prem_df

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,-0.015088,-0.011882,0.030212,-0.024769,0.000206,-0.004252,-0.072456,-0.038330,0.011761,-0.004634,...,-0.010594,0.002403,0.009023,0.002165,0.006261,0.002252,0.006014,0.003821,0.003957,-0.010862
1,0.026002,0.032716,0.008117,0.004714,-0.009129,0.041783,-0.072746,0.033849,0.016534,0.003710,...,0.007227,0.035797,-0.000040,0.063216,-0.051918,0.009105,-0.005771,0.003806,-0.039854,0.049559
2,0.031432,-0.007518,0.062258,0.014156,0.017808,-0.003648,0.097004,-0.004160,0.006059,0.002259,...,0.018558,0.008678,0.003168,0.002547,0.015856,-0.021336,0.007165,0.049768,0.033823,0.049995
3,0.012755,-0.007574,0.022808,-0.005224,0.031377,0.022595,0.062643,0.009672,0.012504,-0.001473,...,-0.018234,0.011484,0.013229,0.018020,0.008921,0.016580,0.001391,0.018208,0.009987,0.034374
4,0.045635,0.012002,-0.067054,0.001002,-0.026486,0.034046,0.006024,-0.004849,0.007666,-0.024157,...,0.018491,0.010403,-0.000356,0.030371,0.013766,0.018221,-0.000326,-0.020112,-0.004673,0.049224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.000990,0.021250,0.009363,0.011190,0.000242,0.034339,0.019135,0.000760,0.004438,0.005749,...,0.047939,0.001597,0.001221,0.001353,0.021475,0.013362,0.002058,0.000000,0.008584,0.007080
1507,0.002308,-0.014688,-0.035250,-0.010060,-0.003867,0.012294,0.016327,0.002658,0.032401,-0.000336,...,0.010032,-0.013335,0.004180,-0.016216,-0.009811,0.008439,-0.000790,-0.047912,0.000000,0.003047
1508,-0.002303,-0.004348,-0.007692,0.013720,-0.011160,-0.011861,0.012450,-0.001515,0.013442,0.009082,...,-0.004370,-0.008961,-0.002428,-0.010989,-0.007077,0.014121,-0.007430,0.002581,0.014894,-0.018925
1509,0.025387,0.016843,0.007752,-0.005013,0.006133,0.012147,0.015470,-0.002275,0.011348,0.018667,...,-0.012370,-0.002965,0.000348,0.008333,0.007128,0.005157,0.018474,0.000000,-0.009434,-0.014051


In [21]:
mkt_risk_prem_df = pd.DataFrame(ffdata['Market Returns'] - ffdata['Risk-free rate'])
mkt_risk_prem_df.rename(columns={0:"Market Risk Premium"}, inplace= True)
mkt_risk_prem_df = mkt_risk_prem_df.reindex(eqt_risk_prem_df.index)
mkt_risk_prem_df

,Market Risk Premium
0,-0.00084
1,0.01216
2,0.00186
3,0.00266
4,0.00476
...,...
1506,0.00520
1507,0.00100
1508,-0.00090
1509,-0.00070


$$ \hat{\sigma}^{2}_{p} = Var(\tilde{r}_{p}) = w^{T}\hat{\beta}\hat{\sigma}^{2}_{M}\hat{\beta}^{T}w+w^{T}\hat{\Delta}w$$

In [129]:
def var_portfolio(w, b, m, d):
    return np.dot(np.dot(np.dot(np.dot(np.transpose(w), b), m), np.transpose(b)),
                  w) + np.dot(np.dot(np.transpose(w), d), w)
#((((w.T @ beta) @ var) @ beta.T) @ w)
#np.dot(np.dot(np.dot(np.dot(np.transpose(w), b), m), np.transpose(b)), w) + np.dot(np.dot(np.transpose(w), d), w)
#(((((np.array(w).T @ np.array(b)) @ np.array(m)) @ np.array(b).T) @ np.array(w)) + 
            #((np.array(w).T @ np.array(d)) @ np.array(w)))

### Rolling Window 504

#### i) Use OLS to estimate the market betas for each stock:

In [116]:
betas_q3_504 = np.zeros(shape=(1007,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_beta = []
    for i in range(1007):
        model = OLS(ri_minus_rf[i:i+503], add_constant(rm_minus_rf[i:i+503]))
        res = model.fit()
        beta = res.params[1]
        col_beta.append(beta)
    betas_q3_504 = np.c_[betas_q3_504, col_beta]
betas_q3_504

array([[0.        , 1.58613189, 1.40039744, ..., 2.1813825 , 1.60233067,
        2.27424324],
       [0.        , 1.58505739, 1.39895521, ..., 2.18106146, 1.60265967,
        2.27349407],
       [0.        , 1.56852779, 1.38326599, ..., 2.15678101, 1.61669276,
        2.2867706 ],
       ...,
       [0.        , 1.04940178, 1.71999022, ..., 1.42024318, 1.07523025,
        1.69069858],
       [0.        , 1.04934783, 1.71996254, ..., 1.42025387, 1.07547457,
        1.69115285],
       [0.        , 1.04899455, 1.72005751, ..., 1.42046858, 1.07501208,
        1.68990684]])

In [117]:
betas_df_q3_504 = pd.DataFrame(betas_q3_504).drop(0, axis = 1)
betas_df_q3_504

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,1.586132,1.400397,2.201703,1.143659,0.984847,1.567119,1.805154,0.771245,1.559725,0.864650,...,0.922545,1.163089,1.070224,2.305267,1.510690,1.440079,1.358351,2.181382,1.602331,2.274243
1,1.585057,1.398955,2.201317,1.139775,0.979738,1.564692,1.801294,0.769244,1.564287,0.864417,...,0.922761,1.163957,1.070103,2.303939,1.508571,1.441680,1.357632,2.181061,1.602660,2.273494
2,1.568528,1.383266,2.199448,1.143749,0.980552,1.562290,1.839123,0.762197,1.543885,0.861147,...,0.907057,1.158397,1.077900,2.256219,1.529209,1.438060,1.348285,2.156781,1.616693,2.286771
3,1.565407,1.383734,2.198504,1.141111,0.979743,1.561133,1.843936,0.760247,1.543489,0.863790,...,0.911997,1.158007,1.075599,2.253799,1.527147,1.442900,1.354719,2.155936,1.614397,2.293199
4,1.563436,1.384451,2.197515,1.141732,0.976663,1.560020,1.838144,0.760024,1.542143,0.865187,...,0.912934,1.157517,1.075112,2.250480,1.526579,1.442121,1.354571,2.154420,1.613581,2.290404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,1.049304,1.718971,1.326889,1.149744,0.822715,0.948279,1.031868,0.537028,1.998436,0.721880,...,0.883524,1.774767,0.598938,1.442129,1.075951,0.732726,1.379887,1.423299,1.076361,1.692562
1003,1.049207,1.719024,1.326736,1.149722,0.822565,0.948359,1.033234,0.536986,1.998578,0.722371,...,0.883948,1.774756,0.599008,1.442054,1.073838,0.732854,1.380064,1.423244,1.076745,1.692292
1004,1.049402,1.719990,1.325560,1.148348,0.822522,0.949578,1.031047,0.537483,1.995442,0.720507,...,0.885010,1.774997,0.599195,1.440668,1.072807,0.731815,1.379589,1.420243,1.075230,1.690699
1005,1.049348,1.719963,1.325524,1.148276,0.822531,0.949600,1.031096,0.537497,1.995745,0.720484,...,0.885050,1.775171,0.599277,1.440606,1.072827,0.731823,1.379788,1.420254,1.075475,1.691153


#### ii) Estimate the variance of the market, $\hat{\sigma}^{2}_{M}=Var(\tilde{r}_M)$ and the idiosyncratic variance,  $\hat{\sigma}^{2}_{i}=Var(\tilde{\epsilon}_i)$, of each security in your portfolio.

In [108]:
var_rm_q3_504 = list(ffdata['Market Returns'].rolling(504).var().dropna())
var_rm_q3_504 = var_rm_q3_504[:-1]
var_rm_q3_504, len(var_rm_q3_504)

([4.601952270503969e-05,
  4.651710625138069e-05,
  4.628661361197901e-05,
  4.628317330146116e-05,
  4.6442464577613765e-05,
  4.643105332291971e-05,
  4.634335636971829e-05,
  4.6310652119031926e-05,
  4.6355279395058304e-05,
  4.6263776704850346e-05,
  4.6293471512196744e-05,
  4.625182618574277e-05,
  4.634404426614923e-05,
  4.6822919625106595e-05,
  4.681742020101622e-05,
  4.683898509735249e-05,
  4.6646033639433334e-05,
  4.660696819085497e-05,
  4.627312123292004e-05,
  4.6274949312064224e-05,
  4.628111233393301e-05,
  4.626996272365815e-05,
  4.6428645271229864e-05,
  4.6272909053614916e-05,
  4.62715080942915e-05,
  4.611079518918246e-05,
  4.6188838043169655e-05,
  4.613817838997769e-05,
  4.594301618858289e-05,
  4.595107766101812e-05,
  4.6048522160686776e-05,
  4.5886758023288935e-05,
  4.594706424153502e-05,
  4.588421329167859e-05,
  4.587302076430316e-05,
  4.5890042443750286e-05,
  4.5908654580453875e-05,
  4.5870437020732834e-05,
  4.587483760137596e-05,
  4.608390

In [50]:
var_e_q3_504 = np.zeros(shape=(1007,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_vars = []
    for i in range(1007):
        model = OLS(ri_minus_rf[i:i+503], add_constant(rm_minus_rf[i:i+503]))
        res = model.fit()
        varis = res.resid
        col_vars.append(np.var(varis))
    var_e_q3_504 = np.c_[var_e_q3_504, col_vars]
var_e_q3_504

array([[0.        , 0.00032241, 0.00016527, ..., 0.00090192, 0.00079214,
        0.00088757],
       [0.        , 0.00032204, 0.00016512, ..., 0.00090189, 0.00079207,
        0.00088745],
       [0.        , 0.00032268, 0.0001648 , ..., 0.00090647, 0.00078628,
        0.00088911],
       ...,
       [0.        , 0.00027802, 0.00092841, ..., 0.00367989, 0.00115919,
        0.00199991],
       [0.        , 0.00027791, 0.00092886, ..., 0.00368292, 0.00115733,
        0.00199361],
       [0.        , 0.00027757, 0.00092886, ..., 0.0036828 , 0.00115735,
        0.00198985]])

In [52]:
vars_df_q3_504 = pd.DataFrame(var_e_q3_504).drop(0, axis = 1)
vars_df_q3_504

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.000322,0.000165,0.000687,0.000422,0.000270,0.000510,0.000968,0.000170,0.000248,0.000286,...,0.000863,0.000088,0.000180,0.001600,0.000720,0.000383,0.000311,0.000902,0.000792,0.000888
1,0.000322,0.000165,0.000685,0.000421,0.000271,0.000510,0.000958,0.000167,0.000249,0.000286,...,0.000863,0.000088,0.000180,0.001600,0.000720,0.000383,0.000311,0.000902,0.000792,0.000887
2,0.000323,0.000165,0.000685,0.000421,0.000271,0.000510,0.000940,0.000166,0.000251,0.000286,...,0.000864,0.000088,0.000179,0.001601,0.000711,0.000383,0.000312,0.000906,0.000786,0.000889
3,0.000321,0.000165,0.000679,0.000421,0.000271,0.000510,0.000929,0.000166,0.000251,0.000286,...,0.000865,0.000088,0.000180,0.001601,0.000711,0.000382,0.000314,0.000903,0.000784,0.000889
4,0.000322,0.000165,0.000679,0.000421,0.000269,0.000510,0.000922,0.000166,0.000251,0.000287,...,0.000867,0.000088,0.000180,0.001605,0.000711,0.000382,0.000314,0.000903,0.000784,0.000887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1002,0.000278,0.000929,0.003222,0.000736,0.000539,0.000473,0.001003,0.000275,0.002759,0.000623,...,0.001070,0.001041,0.000408,0.001742,0.001739,0.000740,0.001347,0.003685,0.001164,0.002005
1003,0.000278,0.000928,0.003221,0.000733,0.000539,0.000473,0.001009,0.000275,0.002759,0.000624,...,0.001071,0.001041,0.000408,0.001741,0.001759,0.000740,0.001345,0.003685,0.001161,0.002001
1004,0.000278,0.000928,0.003220,0.000731,0.000539,0.000475,0.001006,0.000275,0.002754,0.000621,...,0.001075,0.001041,0.000408,0.001740,0.001759,0.000739,0.001345,0.003680,0.001159,0.002000
1005,0.000278,0.000929,0.003222,0.000732,0.000539,0.000475,0.001006,0.000275,0.002755,0.000621,...,0.001075,0.001039,0.000408,0.001741,0.001759,0.000739,0.001343,0.003683,0.001157,0.001994


#### iii) Using the market capitalization weights (from the last day in the rolling window)of your securities, estimate the variance and standard deviation of your portfolio.

Formula: 
$$ \hat{\sigma}^{2}_{p} = Var(\tilde{r}_{p}) = w^{T}\hat{\beta}\hat{\sigma}^{2}_{M}\hat{\beta}^{T}w+w^{T}\hat{\Delta}w$$

In [99]:
df_weights_504 = df_weights.tail(1007)
df_weights_504

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
503,0.018565,0.027973,0.001530,0.000238,0.005111,0.066085,0.000360,0.009388,0.000279,0.000975,...,0.000729,0.002962,0.104930,0.001076,0.004838,0.001353,0.000424,0.004341,0.000428,0.000281
504,0.018351,0.027784,0.001531,0.000238,0.005027,0.067496,0.000358,0.009413,0.000273,0.000961,...,0.000709,0.002988,0.105030,0.001049,0.004762,0.001345,0.000413,0.004189,0.000421,0.000296
505,0.018299,0.027789,0.001563,0.000236,0.005035,0.067324,0.000378,0.009300,0.000273,0.000973,...,0.000730,0.002992,0.103962,0.001042,0.004751,0.001369,0.000427,0.004262,0.000422,0.000311
506,0.018792,0.028031,0.001533,0.000237,0.005063,0.066842,0.000377,0.009191,0.000277,0.000950,...,0.000758,0.002972,0.102527,0.001087,0.004778,0.001354,0.000431,0.004297,0.000423,0.000308
507,0.018690,0.027641,0.001549,0.000233,0.005003,0.067839,0.000374,0.008993,0.000275,0.000930,...,0.000768,0.002985,0.100511,0.001067,0.004920,0.001548,0.000442,0.004280,0.000423,0.000305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [133]:
var_port_q3_504 = []
for i in range(len(df_weights_504)):
    diag_mat = np.diag(vars_df_q3_504.iloc[i])
    res = var_portfolio(df_weights_504.iloc[i], betas_df_q3_504.iloc[i], var_rm_q3_504[i], diag_mat)
    var_port_q3_504.append(res)
arr_var_q3_504 = np.array(var_port_q3_504)
arr_var_q3_504

array([6.63549209e-05, 6.72981929e-05, 6.70513956e-05, ...,
       4.47347672e-04, 4.46184599e-04, 4.46173349e-04])

In [134]:
arr_sd_q3_504 = np.sqrt(arr_var_q3_504)
arr_sd_q3_504

array([0.00814585, 0.00820355, 0.00818849, ..., 0.0211506 , 0.02112308,
       0.02112282])

#### iv) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [131]:
#getting one-day ahead returns array
dayahead504_port_ret_q3 = []
dayahead504_ret_q3 = portfolio_q2_ret.loc[504:1510].to_numpy()
dayahead504_w_q3 = weights_q2_504.loc[504:1510].to_numpy()
for i in range(0, len(dayahead504_w_q2)):
    dayahead504_port_ret_q3.append(np.multiply(dayahead504_ret_q3[i], dayahead504_w_q3[i]))
dayahead504_port_ret_q3_arr = np.sum(np.array(dayahead504_port_ret_q3), axis = 1)
dayahead504_port_ret_q3_arr

array([ 0.01826034,  0.00564215, -0.00055066, ..., -0.00414611,
        0.00514476, -0.00946444])

#### v) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [147]:
standardized_outcomes_504_q3 = dayahead504_port_ret_q3_arr / arr_sd_q3_504
std_outcomes_504_q3 = pd.DataFrame(standardized_outcomes_504_q3)
std_outcomes_504_q3.index += 504
std_outcomes_504_q3.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_504_q3

,Standardized Outcome
504,2.241673
505,0.687769
506,-0.067248
507,1.343726
508,0.453598
...,...
1506,0.572534
1507,0.245258
1508,-0.196028
1509,0.243561


### Rolling Window 252

#### i) Use OLS to estimate the market betas for each stock:

In [137]:
betas_q3_252 = np.zeros(shape=(1259,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_beta = []
    for i in range(1259):
        model = OLS(ri_minus_rf[i:i+251], add_constant(rm_minus_rf[i:i+251]))
        res = model.fit()
        beta = res.params[1]
        col_beta.append(beta)
    betas_q3_252 = np.c_[betas_q3_252, col_beta]
betas_q3_252

array([[0.        , 1.9319157 , 1.56218804, ..., 2.6725418 , 1.81207486,
        2.30861338],
       [0.        , 1.93020096, 1.56099466, ..., 2.67307771, 1.81531823,
        2.3086432 ],
       [0.        , 1.91678797, 1.5441574 , ..., 2.66836787, 1.85087787,
        2.30164876],
       ...,
       [0.        , 1.13115708, 1.93123882, ..., 1.57264432, 1.17097134,
        1.68377417],
       [0.        , 1.12920755, 1.93157658, ..., 1.57335547, 1.17020914,
        1.67916135],
       [0.        , 1.14018041, 1.91684191, ..., 1.58306463, 1.16520439,
        1.69638467]])

In [138]:
betas_df_q3_252 = pd.DataFrame(betas_q3_252).drop(0, axis = 1)
betas_df_q3_252

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,1.931916,1.562188,2.317098,1.147359,0.955836,1.385805,2.060276,0.899883,1.342231,0.841983,...,0.701275,1.216221,1.061192,2.094228,1.814111,1.432960,1.147995,2.672542,1.812075,2.308613
1,1.930201,1.560995,2.321713,1.143028,0.955016,1.386188,2.056838,0.895685,1.346941,0.844325,...,0.701769,1.216176,1.062287,2.093458,1.816599,1.431876,1.150219,2.673078,1.815318,2.308643
2,1.916788,1.544157,2.322160,1.195700,0.968974,1.372672,2.155587,0.874056,1.352562,0.834889,...,0.726037,1.198762,1.063314,2.055826,1.860771,1.443881,1.167527,2.668368,1.850878,2.301649
3,1.913509,1.542065,2.354886,1.184452,0.960986,1.348913,2.160270,0.871213,1.352927,0.827997,...,0.742748,1.199612,1.065194,2.107041,1.869526,1.464941,1.146697,2.722915,1.827078,2.339606
4,1.905218,1.541584,2.366488,1.183212,0.951547,1.339436,2.155961,0.868018,1.363342,0.832413,...,0.758065,1.206839,1.067783,2.130073,1.863260,1.473049,1.163904,2.760195,1.858593,2.347110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,1.129996,1.931494,1.456411,1.198005,0.573958,0.833084,0.977473,0.467603,2.888859,0.817335,...,0.878836,2.215995,0.463726,1.857955,1.022083,0.787456,1.272878,1.572274,1.171988,1.683935
1255,1.130210,1.932652,1.455659,1.196772,0.573242,0.833962,0.980778,0.467705,2.887693,0.818818,...,0.879974,2.215311,0.463939,1.859770,1.018341,0.788376,1.271248,1.571422,1.172396,1.684099
1256,1.131157,1.931239,1.455531,1.197119,0.573145,0.835204,0.977920,0.467129,2.889860,0.819207,...,0.882000,2.214511,0.463860,1.860776,1.019610,0.787770,1.268903,1.572644,1.170971,1.683774
1257,1.129208,1.931577,1.453296,1.198668,0.573467,0.835939,0.976104,0.466884,2.887394,0.817843,...,0.881068,2.215426,0.463385,1.861628,1.015855,0.784523,1.271422,1.573355,1.170209,1.679161


#### ii) Estimate the variance of the market, $\hat{\sigma}^{2}_{M}=Var(\tilde{r}_M)$ and the idiosyncratic variance,  $\hat{\sigma}^{2}_{i}=Var(\tilde{\epsilon}_i)$, of each security in your portfolio.

In [139]:
var_rm_q3_252 = list(ffdata['Market Returns'].rolling(252).var().dropna())
var_rm_q3_252 = var_rm_q3_252[:-1]
var_rm_q3_252, len(var_rm_q3_252)

([5.00546569278442e-05,
  5.049132406880417e-05,
  5.063620502118512e-05,
  5.079488316574972e-05,
  5.0810256118383625e-05,
  5.075544472902045e-05,
  5.064044820717134e-05,
  5.072227834692976e-05,
  5.068556757098592e-05,
  5.069892035034467e-05,
  5.0837047366091214e-05,
  5.097263264402708e-05,
  5.1291223676721704e-05,
  5.1411035698475954e-05,
  5.146795089483339e-05,
  5.1564262790109426e-05,
  5.116043255549233e-05,
  5.100625482198193e-05,
  5.0179867672168484e-05,
  5.021428998292546e-05,
  5.054503256814016e-05,
  5.0687419686334043e-05,
  5.073656232214003e-05,
  5.034946104471006e-05,
  5.076353569847596e-05,
  4.999692831847216e-05,
  4.9997599443495875e-05,
  5.0225226554101076e-05,
  4.989830946057043e-05,
  5.002678935053439e-05,
  4.9900513501549374e-05,
  4.956247438816166e-05,
  4.9473345190665926e-05,
  4.9567699203187273e-05,
  4.950778157212422e-05,
  5.0181766110162545e-05,
  5.025725083791819e-05,
  5.037231818756721e-05,
  5.070487130841714e-05,
  5.085230506

In [140]:
var_e_q3_252 = np.zeros(shape=(1259,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_vars = []
    for i in range(1259):
        model = OLS(ri_minus_rf[i:i+251], add_constant(rm_minus_rf[i:i+251]))
        res = model.fit()
        varis = res.resid
        col_vars.append(np.var(varis))
    var_e_q3_252 = np.c_[var_e_q3_252, col_vars]
var_e_q3_252

array([[0.        , 0.00039619, 0.00020445, ..., 0.00094436, 0.00054574,
        0.00077488],
       [0.        , 0.00039566, 0.00020412, ..., 0.00094415, 0.00054714,
        0.00077478],
       [0.        , 0.00039614, 0.00020326, ..., 0.0009464 , 0.00053506,
        0.00077391],
       ...,
       [0.        , 0.00027243, 0.00088567, ..., 0.0024766 , 0.00078792,
        0.00138948],
       [0.        , 0.00026971, 0.00088678, ..., 0.00248556, 0.00078751,
        0.00137423],
       [0.        , 0.00026425, 0.00087663, ..., 0.00248117, 0.00078773,
        0.001363  ]])

In [141]:
vars_df_q3_252 = pd.DataFrame(var_e_q3_252).drop(0, axis = 1)
vars_df_q3_252

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.000396,0.000204,0.000711,0.000472,0.000259,0.000552,0.001117,0.000185,0.000216,0.000196,...,0.001164,0.000103,0.000138,0.001794,0.000485,0.000392,0.000393,0.000944,0.000546,0.000775
1,0.000396,0.000204,0.000707,0.000470,0.000259,0.000552,0.001101,0.000179,0.000218,0.000198,...,0.001165,0.000103,0.000138,0.001795,0.000485,0.000393,0.000393,0.000944,0.000547,0.000775
2,0.000396,0.000203,0.000707,0.000481,0.000258,0.000551,0.001063,0.000177,0.000218,0.000199,...,0.001167,0.000101,0.000138,0.001788,0.000467,0.000392,0.000392,0.000946,0.000535,0.000774
3,0.000393,0.000203,0.000699,0.000481,0.000257,0.000553,0.001029,0.000177,0.000218,0.000199,...,0.001167,0.000101,0.000138,0.001797,0.000467,0.000391,0.000393,0.000951,0.000532,0.000772
4,0.000393,0.000203,0.000701,0.000481,0.000254,0.000552,0.001016,0.000177,0.000220,0.000200,...,0.001167,0.000102,0.000138,0.001806,0.000467,0.000392,0.000397,0.000974,0.000549,0.000771
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1254,0.000274,0.000891,0.002006,0.000372,0.000411,0.000232,0.000821,0.000233,0.002552,0.000471,...,0.001266,0.000623,0.000345,0.001524,0.000948,0.000859,0.000721,0.002479,0.000790,0.001390
1255,0.000274,0.000888,0.002007,0.000371,0.000410,0.000230,0.000833,0.000233,0.002547,0.000474,...,0.001268,0.000623,0.000344,0.001513,0.000988,0.000859,0.000713,0.002478,0.000790,0.001389
1256,0.000272,0.000886,0.002007,0.000371,0.000410,0.000233,0.000824,0.000232,0.002542,0.000474,...,0.001276,0.000623,0.000344,0.001512,0.000989,0.000858,0.000709,0.002477,0.000788,0.001389
1257,0.000270,0.000887,0.002008,0.000370,0.000410,0.000233,0.000823,0.000232,0.002542,0.000472,...,0.001275,0.000624,0.000344,0.001513,0.000979,0.000851,0.000705,0.002486,0.000788,0.001374


#### iii) Using the market capitalization weights (from the last day in the rolling window)of your securities, estimate the variance and standard deviation of your portfolio.

Formula: 
$$ \hat{\sigma}^{2}_{p} = Var(\tilde{r}_{p}) = w^{T}\hat{\beta}\hat{\sigma}^{2}_{M}\hat{\beta}^{T}w+w^{T}\hat{\Delta}w$$

In [142]:
df_weights_252 = df_weights.tail(1259)
df_weights_252

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
251,0.014672,0.034202,0.001147,0.000248,0.004016,0.032580,0.000359,0.007714,0.000256,0.000820,...,0.000620,0.002512,0.101078,0.001468,0.002760,0.001764,0.000400,0.002202,0.000444,0.000520
252,0.014634,0.033957,0.001150,0.000233,0.004031,0.032230,0.000350,0.007694,0.000253,0.000834,...,0.000607,0.002495,0.101953,0.001438,0.002811,0.001749,0.000398,0.002235,0.000449,0.000503
253,0.014417,0.033738,0.001083,0.000234,0.004051,0.032946,0.000339,0.007724,0.000252,0.000840,...,0.000599,0.002485,0.101522,0.001338,0.002748,0.001711,0.000405,0.002051,0.000452,0.000472
254,0.014450,0.033630,0.001037,0.000234,0.004070,0.033326,0.000330,0.007730,0.000244,0.000832,...,0.000587,0.002433,0.100190,0.001250,0.002753,0.001670,0.000390,0.001859,0.000418,0.000454
255,0.014070,0.033624,0.001045,0.000230,0.004048,0.033202,0.000328,0.007761,0.000248,0.000831,...,0.000582,0.002430,0.100672,0.001220,0.002745,0.001674,0.000385,0.001789,0.000405,0.000459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [143]:
var_port_q3_252 = []
for i in range(len(df_weights_252)):
    diag_mat = np.diag(vars_df_q3_252.iloc[i])
    res = var_portfolio(df_weights_252.iloc[i], betas_df_q3_252.iloc[i], var_rm_q3_252[i], diag_mat)
    var_port_q3_252.append(res)
arr_var_q3_252 = np.array(var_port_q3_252)
arr_var_q3_252

array([7.10001196e-05, 7.15088950e-05, 7.13106077e-05, ...,
       2.80209391e-04, 2.79455380e-04, 2.78235687e-04])

In [144]:
arr_sd_q3_252 = np.sqrt(arr_var_q3_252)
arr_sd_q3_252

array([0.00842616, 0.00845629, 0.00844456, ..., 0.01673946, 0.01671692,
       0.0166804 ])

#### iv) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [145]:
#getting one-day ahead returns array
dayahead252_port_ret_q3 = []
dayahead252_ret_q3 = portfolio_q2_ret.loc[252:1510].to_numpy()
dayahead252_w_q3 = weights_q2_252.loc[252:1510].to_numpy()
for i in range(0, len(dayahead252_w_q2)):
    dayahead252_port_ret_q3.append(np.multiply(dayahead252_ret_q3[i], dayahead252_w_q3[i]))
dayahead252_port_ret_q3_arr = np.sum(np.array(dayahead252_port_ret_q3), axis = 1)
dayahead252_port_ret_q3_arr

array([-0.00643605, -0.01146078, -0.00250408, ..., -0.00414611,
        0.00514476, -0.00946444])

#### v) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [146]:
standardized_outcomes_252_q3 = dayahead252_port_ret_q3_arr / arr_sd_q3_252
std_outcomes_252_q3 = pd.DataFrame(standardized_outcomes_252_q3)
std_outcomes_252_q3.index += 252
std_outcomes_252_q3.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_252_q3

,Standardized Outcome
252,-0.763818
253,-1.355296
254,-0.296531
255,0.545255
256,0.646344
...,...
1506,0.722503
1507,0.309746
1508,-0.247685
1509,0.307758


### Rolling Window 126

#### i) Use OLS to estimate the market betas for each stock:

In [148]:
betas_q3_126 = np.zeros(shape=(1385,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_beta = []
    for i in range(1385):
        model = OLS(ri_minus_rf[i:i+125], add_constant(rm_minus_rf[i:i+125]))
        res = model.fit()
        beta = res.params[1]
        col_beta.append(beta)
    betas_q3_126 = np.c_[betas_q3_126, col_beta]
betas_q3_126

array([[0.        , 1.8587196 , 1.74873984, ..., 2.68095759, 1.51510429,
        2.36746127],
       [0.        , 1.85954524, 1.74715317, ..., 2.67584122, 1.51969958,
        2.37150663],
       [0.        , 1.87006592, 1.71116045, ..., 2.74035345, 1.61722432,
        2.33411603],
       ...,
       [0.        , 1.18005442, 1.8923816 , ..., 1.65167118, 0.98460477,
        2.01873032],
       [0.        , 1.181316  , 1.8953813 , ..., 1.62771547, 0.96961463,
        2.03000054],
       [0.        , 1.17989873, 1.89854685, ..., 1.6231458 , 0.95574677,
        2.03780855]])

In [149]:
betas_df_q3_126 = pd.DataFrame(betas_q3_126).drop(0, axis = 1)
betas_df_q3_126

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,1.858720,1.748740,2.090839,0.863111,0.797002,1.536315,2.082316,0.950925,1.132253,0.674799,...,-0.037736,1.274847,0.896070,2.508117,1.622644,1.251564,1.097225,2.680958,1.515104,2.367461
1,1.859545,1.747153,2.090576,0.861535,0.799008,1.546799,2.076093,0.945995,1.132293,0.674788,...,-0.033140,1.276174,0.894386,2.501367,1.622455,1.256673,1.094990,2.675841,1.519700,2.371507
2,1.870066,1.711160,2.140906,0.853506,0.832043,1.503603,2.209823,0.932673,1.136640,0.706417,...,0.017190,1.259573,0.926948,2.483351,1.751218,1.278058,1.102598,2.740353,1.617224,2.334116
3,1.860781,1.719649,2.121365,0.852611,0.824514,1.499804,2.184585,0.932889,1.142616,0.707710,...,0.017777,1.250684,0.925417,2.488452,1.742482,1.284513,1.099942,2.724879,1.609754,2.318518
4,1.855515,1.710480,2.139003,0.877110,0.857206,1.489830,2.152956,0.922099,1.178655,0.740507,...,0.111562,1.232636,0.921892,2.441808,1.744600,1.310097,1.102684,2.765111,1.626875,2.335174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,1.177943,1.872456,1.285212,0.903495,0.548774,0.877056,1.121534,0.447239,1.635484,0.869361,...,0.941610,1.685746,0.354056,1.843567,1.027747,0.886456,1.415332,1.650789,0.995875,2.017475
1381,1.178430,1.873308,1.284081,0.902287,0.547696,0.878505,1.132335,0.446102,1.637488,0.874425,...,0.946875,1.687885,0.355004,1.843211,1.014340,0.887604,1.421884,1.648170,0.995835,2.018157
1382,1.180054,1.892382,1.269821,0.900144,0.550474,0.888848,1.134361,0.449499,1.629157,0.877462,...,0.956372,1.689515,0.353743,1.831304,1.025718,0.897379,1.413123,1.651671,0.984605,2.018730
1383,1.181316,1.895381,1.273381,0.885996,0.552735,0.893454,1.139273,0.439229,1.626673,0.876230,...,0.951660,1.698430,0.353187,1.842794,1.031650,0.901136,1.429866,1.627715,0.969615,2.030001


#### ii) Estimate the variance of the market, $\hat{\sigma}^{2}_{M}=Var(\tilde{r}_M)$ and the idiosyncratic variance,  $\hat{\sigma}^{2}_{i}=Var(\tilde{\epsilon}_i)$, of each security in your portfolio.

In [150]:
var_rm_q3_126 = list(ffdata['Market Returns'].rolling(126).var().dropna())
var_rm_q3_126 = var_rm_q3_126[:-1]
var_rm_q3_126, len(var_rm_q3_126)

([5.677102984126982e-05,
  5.74354641269841e-05,
  5.6288599999999975e-05,
  5.6952635555555524e-05,
  5.697653079365077e-05,
  5.678806603174601e-05,
  5.6477554285714266e-05,
  5.631318412698411e-05,
  5.623261714285712e-05,
  5.597202603174601e-05,
  5.596053079365077e-05,
  5.6019149841269824e-05,
  5.758051555555554e-05,
  5.7309399365079346e-05,
  5.803026857142855e-05,
  5.8169119999999986e-05,
  5.8134450793650776e-05,
  5.759706412698411e-05,
  5.6427509841269816e-05,
  5.642999428571426e-05,
  5.653655428571426e-05,
  5.6796577142857116e-05,
  5.6829034285714256e-05,
  5.804937714285711e-05,
  5.9706037460317435e-05,
  5.795551555555553e-05,
  5.94367441269841e-05,
  5.918790603174601e-05,
  5.923559936507934e-05,
  5.9165399365079344e-05,
  6.06546241269841e-05,
  5.989579936507934e-05,
  6.117797650793648e-05,
  6.10408641269841e-05,
  6.152750984126982e-05,
  6.142502857142855e-05,
  6.142585714285713e-05,
  6.172903936507934e-05,
  6.164881269841267e-05,
  6.1704297142857

In [151]:
var_e_q3_126 = np.zeros(shape=(1385,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_vars = []
    for i in range(1385):
        model = OLS(ri_minus_rf[i:i+125], add_constant(rm_minus_rf[i:i+125]))
        res = model.fit()
        varis = res.resid
        col_vars.append(np.var(varis))
    var_e_q3_126 = np.c_[var_e_q3_126, col_vars]
var_e_q3_126

array([[0.        , 0.00033091, 0.00023457, ..., 0.00089622, 0.00061017,
        0.00083786],
       [0.        , 0.00033016, 0.00023392, ..., 0.00089916, 0.00061103,
        0.00083976],
       [0.        , 0.0003314 , 0.00023348, ..., 0.00089376, 0.00058434,
        0.00083603],
       ...,
       [0.        , 0.00014199, 0.00045503, ..., 0.00159275, 0.00057209,
        0.00125177],
       [0.        , 0.00014195, 0.0004573 , ..., 0.00159915, 0.00056703,
        0.00124892],
       [0.        , 0.00014161, 0.00045658, ..., 0.00159687, 0.00055472,
        0.0012489 ]])

In [152]:
vars_df_q3_126 = pd.DataFrame(var_e_q3_126).drop(0, axis = 1)
vars_df_q3_126

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.000331,0.000235,0.000796,0.000553,0.000191,0.000413,0.001187,0.000131,0.000167,0.000222,...,0.001448,0.000109,0.000134,0.002136,0.000533,0.000517,0.000488,0.000896,0.000610,0.000838
1,0.000330,0.000234,0.000791,0.000548,0.000191,0.000424,0.001148,0.000119,0.000166,0.000222,...,0.001450,0.000109,0.000134,0.002141,0.000533,0.000520,0.000488,0.000899,0.000611,0.000840
2,0.000331,0.000233,0.000789,0.000550,0.000188,0.000421,0.001074,0.000118,0.000166,0.000225,...,0.001463,0.000107,0.000134,0.002141,0.000490,0.000519,0.000488,0.000894,0.000584,0.000836
3,0.000325,0.000237,0.000765,0.000549,0.000188,0.000425,0.001007,0.000118,0.000171,0.000225,...,0.001464,0.000112,0.000134,0.002146,0.000493,0.000513,0.000489,0.000879,0.000576,0.000820
4,0.000325,0.000237,0.000766,0.000551,0.000190,0.000423,0.000982,0.000118,0.000177,0.000229,...,0.001492,0.000113,0.000133,0.002150,0.000493,0.000516,0.000489,0.000887,0.000578,0.000816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1380,0.000145,0.000470,0.001286,0.000267,0.000243,0.000147,0.000671,0.000142,0.001309,0.000265,...,0.001089,0.000240,0.000332,0.000926,0.000537,0.000389,0.000718,0.001594,0.000584,0.001253
1381,0.000142,0.000470,0.001285,0.000267,0.000243,0.000145,0.000701,0.000141,0.001309,0.000271,...,0.001090,0.000227,0.000330,0.000926,0.000556,0.000388,0.000603,0.001594,0.000580,0.001252
1382,0.000142,0.000455,0.001271,0.000266,0.000242,0.000150,0.000702,0.000140,0.001305,0.000271,...,0.001103,0.000227,0.000329,0.000921,0.000555,0.000385,0.000599,0.001593,0.000572,0.001252
1383,0.000142,0.000457,0.001280,0.000262,0.000242,0.000150,0.000703,0.000138,0.001314,0.000271,...,0.001103,0.000226,0.000330,0.000921,0.000555,0.000385,0.000593,0.001599,0.000567,0.001249


#### iii) Using the market capitalization weights (from the last day in the rolling window)of your securities, estimate the variance and standard deviation of your portfolio.

Formula: 
$$ \hat{\sigma}^{2}_{p} = Var(\tilde{r}_{p}) = w^{T}\hat{\beta}\hat{\sigma}^{2}_{M}\hat{\beta}^{T}w+w^{T}\hat{\Delta}w$$

In [153]:
df_weights_126 = df_weights.tail(1385)
df_weights_126

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
125,0.018116,0.037240,0.001599,0.000318,0.004287,0.015935,0.000463,0.008304,0.000257,0.000836,...,0.000481,0.001915,0.092974,0.001375,0.002885,0.001476,0.000379,0.002526,0.000530,0.000463
126,0.017800,0.037490,0.001569,0.000324,0.004315,0.016075,0.000474,0.008177,0.000257,0.000826,...,0.000470,0.001891,0.092188,0.001294,0.002870,0.001477,0.000387,0.002464,0.000529,0.000459
127,0.017727,0.038398,0.001550,0.000328,0.004267,0.015777,0.000474,0.008146,0.000264,0.000832,...,0.000480,0.001843,0.091906,0.001321,0.002820,0.001481,0.000386,0.002442,0.000532,0.000455
128,0.017547,0.038386,0.001501,0.000322,0.004119,0.015748,0.000472,0.008176,0.000255,0.000812,...,0.000454,0.001854,0.091519,0.001327,0.002785,0.001445,0.000384,0.002308,0.000519,0.000437
129,0.017530,0.038236,0.001515,0.000323,0.004116,0.015692,0.000479,0.008088,0.000256,0.000805,...,0.000453,0.001860,0.092027,0.001330,0.002778,0.001433,0.000384,0.002213,0.000521,0.000442
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [154]:
var_port_q3_126 = []
for i in range(len(df_weights_126)):
    diag_mat = np.diag(vars_df_q3_126.iloc[i])
    res = var_portfolio(df_weights_126.iloc[i], betas_df_q3_126.iloc[i], var_rm_q3_126[i], diag_mat)
    var_port_q3_126.append(res)
arr_var_q3_126 = np.array(var_port_q3_126)
arr_var_q3_126

array([6.97628889e-05, 7.02152997e-05, 6.97454828e-05, ...,
       1.08765762e-04, 1.07983400e-04, 1.08278851e-04])

In [155]:
arr_sd_q3_126 = np.sqrt(arr_var_q3_126)
arr_sd_q3_126

array([0.00835242, 0.00837946, 0.00835138, ..., 0.01042908, 0.01039151,
       0.01040571])

#### iv) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [156]:
#getting one-day ahead returns array
dayahead126_port_ret_q3 = []
dayahead126_ret_q3 = portfolio_q2_ret.loc[126:1510].to_numpy()
dayahead126_w_q3 = weights_q2_126.loc[126:1510].to_numpy()
for i in range(0, len(dayahead126_w_q2)):
    dayahead126_port_ret_q3.append(np.multiply(dayahead126_ret_q3[i], dayahead126_w_q3[i]))
dayahead126_port_ret_q3_arr = np.sum(np.array(dayahead126_port_ret_q3), axis = 1)
dayahead126_port_ret_q3_arr

array([-0.01262911,  0.00056176, -0.0061875 , ..., -0.00414611,
        0.00514476, -0.00946444])

#### v) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [158]:
standardized_outcomes_126_q3 = dayahead126_port_ret_q3_arr / arr_sd_q3_126
std_outcomes_126_q3 = pd.DataFrame(standardized_outcomes_126_q3)
std_outcomes_126_q3.index += 126
std_outcomes_126_q3.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_126_q3

,Standardized Outcome
126,-1.512030
127,0.067040
128,-0.740896
129,-0.016961
130,0.009658
...,...
1506,1.164179
1507,0.498764
1508,-0.397552
1509,0.495093


### Rolling Window 63

#### i) Use OLS to estimate the market betas for each stock:

In [159]:
betas_q3_63 = np.zeros(shape=(1448,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_beta = []
    for i in range(1448):
        model = OLS(ri_minus_rf[i:i+62], add_constant(rm_minus_rf[i:i+62]))
        res = model.fit()
        beta = res.params[1]
        col_beta.append(beta)
    betas_q3_63 = np.c_[betas_q3_63, col_beta]
betas_q3_63

array([[0.        , 2.05281222, 1.6314848 , ..., 2.75372606, 1.27649538,
        2.98277006],
       [0.        , 2.02427953, 1.61056927, ..., 2.7743287 , 1.38067433,
        3.00345426],
       [0.        , 2.02989276, 1.61751051, ..., 2.95940282, 1.55512359,
        2.94348151],
       ...,
       [0.        , 1.17241229, 1.90870263, ..., 2.19046177, 0.86162588,
        2.04912328],
       [0.        , 1.17280462, 1.90645963, ..., 2.19202489, 0.85996116,
        2.04474499],
       [0.        , 1.17361875, 1.90209361, ..., 2.19155911, 0.84245876,
        2.10855709]])

In [160]:
betas_df_q3_63 = pd.DataFrame(betas_q3_63).drop(0, axis = 1)
betas_df_q3_63

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,2.052812,1.631485,2.467977,0.149854,0.834601,1.489859,2.200903,0.891679,1.255516,0.744768,...,-0.315272,1.102946,1.030030,2.210882,1.360785,1.193268,1.398360,2.753726,1.276495,2.982770
1,2.024280,1.610569,2.470555,0.134584,0.813522,1.469599,2.103833,0.854639,1.232819,0.729087,...,-0.229045,1.107336,1.039140,2.219811,1.329962,1.204111,1.393227,2.774329,1.380674,3.003454
2,2.029893,1.617511,2.626662,0.150088,0.836575,1.398232,2.441186,0.785010,1.221262,0.726154,...,-0.089639,1.036171,1.092934,2.160679,1.477214,1.261134,1.475099,2.959403,1.555124,2.943482
3,2.002958,1.605841,2.581234,0.202040,0.870995,1.430943,2.490066,0.782768,1.185146,0.707268,...,-0.167862,1.034573,1.087285,2.176167,1.497215,1.271700,1.409464,2.914264,1.590504,2.940288
4,1.986856,1.585203,2.553937,0.221884,0.851413,1.438855,2.489432,0.773026,1.202064,0.697586,...,-0.132105,1.029349,1.091862,2.189674,1.506381,1.336551,1.386151,2.897875,1.581343,2.893509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,1.155123,1.901754,1.162571,0.910696,0.745226,0.835353,1.144480,0.652799,1.932211,0.842453,...,0.960492,1.787768,0.626726,2.011263,1.084712,1.249704,1.669961,2.061096,0.882343,1.957300
1444,1.152748,1.883088,1.184261,0.903087,0.751742,0.834111,1.189235,0.650191,1.972411,0.842808,...,0.950188,1.750430,0.624365,1.998901,1.088509,1.278237,1.637766,2.106696,0.902994,1.988022
1445,1.172412,1.908703,1.270773,0.916972,0.768153,0.839263,1.201479,0.652601,2.109775,0.914254,...,0.934080,1.702679,0.639688,1.992801,1.077241,1.324466,1.758190,2.190462,0.861626,2.049123
1446,1.172805,1.906460,1.270128,0.914495,0.766944,0.837897,1.198084,0.654865,2.110638,0.912225,...,0.928401,1.706282,0.640288,1.987775,1.090476,1.324441,1.760567,2.192025,0.859961,2.044745


#### ii) Estimate the variance of the market, $\hat{\sigma}^{2}_{M}=Var(\tilde{r}_M)$ and the idiosyncratic variance,  $\hat{\sigma}^{2}_{i}=Var(\tilde{\epsilon}_i)$, of each security in your portfolio.

In [161]:
var_rm_q3_63 = list(ffdata['Market Returns'].rolling(63).var().dropna())
var_rm_q3_63 = var_rm_q3_63[:-1]
var_rm_q3_63, len(var_rm_q3_63)

([5.8434905273937535e-05,
  5.943506912442396e-05,
  5.7831889400921654e-05,
  5.80658064516129e-05,
  5.8346810035842297e-05,
  5.8113179723502306e-05,
  5.7599262672811055e-05,
  6.097627240143369e-05,
  6.08163338453661e-05,
  5.99999897593446e-05,
  6.034555043522785e-05,
  5.966281105990783e-05,
  6.335046594982078e-05,
  6.33131797235023e-05,
  6.664884792626728e-05,
  6.661854582693292e-05,
  6.496304147465438e-05,
  6.383350742447518e-05,
  6.465539170506913e-05,
  6.58262263184844e-05,
  6.644691244239632e-05,
  6.75279160266257e-05,
  6.774866359447005e-05,
  6.626358422939068e-05,
  6.729861751152074e-05,
  6.774078341013825e-05,
  7.048802355350742e-05,
  7.163447004608295e-05,
  6.972407578084998e-05,
  6.950336405529954e-05,
  6.917300051203277e-05,
  6.885972862263185e-05,
  6.992598054275473e-05,
  6.960167946748591e-05,
  6.95037788018433e-05,
  6.962149513568868e-05,
  6.992038914490527e-05,
  7.396512032770097e-05,
  7.394442396313364e-05,
  7.446379928315413e-05,
  

In [162]:
var_e_q3_63 = np.zeros(shape=(1448,1))
for col_index in range(eqt_risk_prem_df.shape[1]):
    ri_minus_rf = eqt_risk_prem_df.iloc[:, col_index]
    rm_minus_rf = mkt_risk_prem_df[["Market Risk Premium"]]
    col_vars = []
    for i in range(1448):
        model = OLS(ri_minus_rf[i:i+62], add_constant(rm_minus_rf[i:i+62]))
        res = model.fit()
        varis = res.resid
        col_vars.append(np.var(varis))
    var_e_q3_63 = np.c_[var_e_q3_63, col_vars]
var_e_q3_63

array([[0.00000000e+00, 3.88278167e-04, 2.37273297e-04, ...,
        1.09215757e-03, 3.43703982e-04, 1.21146211e-03],
       [0.00000000e+00, 3.88152295e-04, 2.37854832e-04, ...,
        1.09327570e-03, 4.02094536e-04, 1.21369986e-03],
       [0.00000000e+00, 3.88324918e-04, 2.44060545e-04, ...,
        1.10035869e-03, 3.47754641e-04, 1.21060880e-03],
       ...,
       [0.00000000e+00, 8.87651773e-05, 3.92475094e-04, ...,
        1.01033499e-03, 2.89565134e-04, 1.13675361e-03],
       [0.00000000e+00, 8.87462678e-05, 3.96649166e-04, ...,
        1.04796946e-03, 2.89027862e-04, 1.13307292e-03],
       [0.00000000e+00, 8.87714491e-05, 3.95288783e-04, ...,
        1.04857770e-03, 2.82756791e-04, 9.51713160e-04]])

In [163]:
vars_df_q3_63 = pd.DataFrame(var_e_q3_63).drop(0, axis = 1)
vars_df_q3_63

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,0.000388,0.000237,0.000973,0.000625,0.000196,0.000434,0.001506,0.000115,0.000176,0.000320,...,0.000972,0.000114,0.000137,0.000830,0.000626,0.000715,0.000661,0.001092,0.000344,0.001211
1,0.000388,0.000238,0.000956,0.000614,0.000199,0.000435,0.001449,0.000096,0.000178,0.000321,...,0.001014,0.000114,0.000135,0.000829,0.000631,0.000716,0.000660,0.001093,0.000402,0.001214
2,0.000388,0.000244,0.000970,0.000616,0.000198,0.000428,0.001276,0.000087,0.000179,0.000322,...,0.001074,0.000108,0.000134,0.000820,0.000573,0.000717,0.000652,0.001100,0.000348,0.001211
3,0.000378,0.000244,0.000919,0.000632,0.000203,0.000431,0.001189,0.000087,0.000186,0.000324,...,0.001105,0.000107,0.000134,0.000821,0.000574,0.000705,0.000675,0.001073,0.000346,0.001177
4,0.000379,0.000252,0.000918,0.000634,0.000189,0.000431,0.001156,0.000086,0.000191,0.000325,...,0.001097,0.000106,0.000134,0.000824,0.000576,0.000768,0.000683,0.001072,0.000346,0.001175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,0.000090,0.000399,0.001592,0.000142,0.000184,0.000189,0.000300,0.000104,0.001438,0.000252,...,0.000653,0.000217,0.000157,0.000865,0.000543,0.000306,0.000592,0.001075,0.000300,0.001167
1444,0.000090,0.000392,0.001582,0.000144,0.000183,0.000189,0.000361,0.000104,0.001416,0.000262,...,0.000655,0.000194,0.000157,0.000863,0.000670,0.000297,0.000573,0.001035,0.000296,0.001150
1445,0.000089,0.000392,0.001560,0.000144,0.000182,0.000202,0.000365,0.000104,0.001352,0.000245,...,0.000678,0.000189,0.000156,0.000864,0.000673,0.000292,0.000524,0.001010,0.000290,0.001137
1446,0.000089,0.000397,0.001578,0.000145,0.000182,0.000204,0.000368,0.000103,0.001369,0.000244,...,0.000673,0.000190,0.000157,0.000863,0.000641,0.000292,0.000523,0.001048,0.000289,0.001133


#### iii) Using the market capitalization weights (from the last day in the rolling window)of your securities, estimate the variance and standard deviation of your portfolio.

Formula: 
$$ \hat{\sigma}^{2}_{p} = Var(\tilde{r}_{p}) = w^{T}\hat{\beta}\hat{\sigma}^{2}_{M}\hat{\beta}^{T}w+w^{T}\hat{\Delta}w$$

In [164]:
df_weights_63 = df_weights.tail(1448)
df_weights_63

Ticker #,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
62,0.019876,0.039362,0.001319,0.000330,0.003897,0.013208,0.000505,0.007949,0.000241,0.000860,...,0.000414,0.002093,0.097895,0.003387,0.002877,0.001174,0.000363,0.002730,0.000660,0.000478
63,0.019960,0.040117,0.001373,0.000328,0.003792,0.013172,0.000508,0.007900,0.000238,0.000843,...,0.000435,0.002077,0.097670,0.003487,0.002764,0.001185,0.000363,0.002850,0.000653,0.000479
64,0.019807,0.039649,0.001360,0.000336,0.003860,0.013410,0.000537,0.007826,0.000233,0.000828,...,0.000415,0.002071,0.096851,0.003516,0.002802,0.001185,0.000348,0.002823,0.000671,0.000486
65,0.019982,0.040530,0.001374,0.000334,0.003871,0.013246,0.000521,0.007873,0.000229,0.000840,...,0.000414,0.002080,0.095815,0.003418,0.002774,0.001114,0.000355,0.002832,0.000675,0.000495
66,0.019650,0.038668,0.001337,0.000330,0.003896,0.013063,0.000520,0.007886,0.000228,0.000846,...,0.000427,0.002101,0.095080,0.003397,0.002817,0.001106,0.000363,0.002776,0.000662,0.000508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,0.013604,0.020282,0.000935,0.000440,0.005093,0.236809,0.000689,0.009864,0.000120,0.000597,...,0.001730,0.003768,0.075515,0.001760,0.002576,0.001812,0.000757,0.003521,0.000384,0.002439
1506,0.013457,0.020469,0.000932,0.000440,0.005034,0.242061,0.000694,0.009756,0.000119,0.000594,...,0.001792,0.003730,0.074719,0.001742,0.002600,0.001815,0.000750,0.003479,0.000382,0.002427
1507,0.013420,0.020067,0.000895,0.000433,0.004990,0.243796,0.000701,0.009732,0.000123,0.000591,...,0.001801,0.003662,0.074651,0.001705,0.002562,0.001821,0.000746,0.003296,0.000380,0.002422
1508,0.013447,0.020065,0.000892,0.000441,0.004955,0.241941,0.000713,0.009759,0.000125,0.000599,...,0.001801,0.003645,0.074790,0.001694,0.002555,0.001854,0.000743,0.003318,0.000388,0.002387


In [165]:
var_port_q3_63 = []
for i in range(len(df_weights_63)):
    diag_mat = np.diag(vars_df_q3_63.iloc[i])
    res = var_portfolio(df_weights_63.iloc[i], betas_df_q3_63.iloc[i], var_rm_q3_63[i], diag_mat)
    var_port_q3_63.append(res)
arr_var_q3_63 = np.array(var_port_q3_63)
arr_var_q3_63

array([7.63172963e-05, 8.01567192e-05, 8.07554042e-05, ...,
       1.09152319e-04, 1.08705594e-04, 1.09124494e-04])

In [166]:
arr_sd_q3_63 = np.sqrt(arr_var_q3_63)
arr_sd_q3_63

array([0.00873598, 0.00895303, 0.0089864 , ..., 0.0104476 , 0.0104262 ,
       0.01044627])

#### iv) Using the market capitalization weights and returns (from the day following the last day in the rolling window) of your securities, calculate the one-day ahead return of the portfolio, $\tilde{r}_{p}$.

In [260]:
#getting one-day ahead returns array
dayahead63_port_ret_q3 = []
dayahead63_ret_q3 = portfolio_q2_ret.loc[63:1510].to_numpy()
dayahead63_w_q3 = weights_q2_63.loc[63:1510].to_numpy()
for i in range(0, len(dayahead63_w_q2)):
    dayahead63_port_ret_q3.append(np.multiply(dayahead63_ret_q3[i], dayahead63_w_q3[i]))
dayahead63_port_ret_q3_arr = np.sum(np.array(dayahead63_port_ret_q3), axis = 1)
dayahead63_port_ret_q3_arr

array([ 0.01740121,  0.0116785 , -0.00496981, ..., -0.00414611,
        0.00514476, -0.00946444])

#### v) Calculate the standardized outcome, $\tilde{z}_{p}$, where $\tilde{z}_{p} = \frac{\tilde{r}_{p}}{\hat{\sigma}_{p}}$ where we make the simplifying assumption that $E[\tilde{r}_{p}] = 0$.

In [168]:
standardized_outcomes_63_q3 = dayahead63_port_ret_q3_arr / arr_sd_q3_63
std_outcomes_63_q3 = pd.DataFrame(standardized_outcomes_63_q3)
std_outcomes_63_q3.index += 63
std_outcomes_63_q3.rename(columns={0: "Standardized Outcome"}, inplace = True)
std_outcomes_63_q3

,Standardized Outcome
63,1.991902
64,1.304419
65,-0.553037
66,-0.527825
67,0.198191
...,...
1506,1.133099
1507,0.485674
1508,-0.396848
1509,0.493445


## b) Compute bias statistics.

In [169]:
bias_stat_q3_504 = np.std(standardized_outcomes_504_q3)
bias_stat_q3_252 = np.std(standardized_outcomes_252_q3)
bias_stat_q3_126 = np.std(standardized_outcomes_126_q3)
bias_stat_q3_63 = np.std(standardized_outcomes_63_q3)
print(bias_stat_q3_504, bias_stat_q3_252, bias_stat_q3_126, bias_stat_q3_63)

1.3118478769931892 1.1320711225951727 1.0749500838326524 1.057843577865541


## Question 5

In [239]:
#randomized portfolio indices
stocks = pd.read_csv("fifty_portfolios.csv")
stocks = stocks[0:50]
stocks.head()

,984,1236,505,1235,1552,1732,918,863,169,1157,...,349,1588,1731,616,512,614,1061,1704,593,726
0,1757,1199,1673,1702,1511,86,1428,307,170,159,...,1044,238,1060,1646,1305,1713,1574,917,1871,728
1,1212,1755,1549,1121,193,1270,679,34,1816,1626,...,1247,484,141,32,516,1416,1117,1633,1666,690
2,425,1082,1616,784,1197,1067,215,410,1752,1081,...,1785,1797,341,997,233,673,1355,1451,248,1751
3,1006,110,980,273,682,1473,887,289,989,1341,...,508,792,2,1356,1315,1776,589,1355,629,1105
4,26,1386,766,1435,466,1792,1865,1491,17,79,...,1577,1685,1088,247,74,1286,147,666,672,1874


In [240]:
best_rolling_window_q3 = 63
print("Best Rolling Window Market Model:", 63)

Best Rolling Window Market Model: 63


In [241]:
betas_df_q3_63

,1,2,3,4,5,6,7,8,9,10,...,91,92,93,94,95,96,97,98,99,100
0,2.052812,1.631485,2.467977,0.149854,0.834601,1.489859,2.200903,0.891679,1.255516,0.744768,...,-0.315272,1.102946,1.030030,2.210882,1.360785,1.193268,1.398360,2.753726,1.276495,2.982770
1,2.024280,1.610569,2.470555,0.134584,0.813522,1.469599,2.103833,0.854639,1.232819,0.729087,...,-0.229045,1.107336,1.039140,2.219811,1.329962,1.204111,1.393227,2.774329,1.380674,3.003454
2,2.029893,1.617511,2.626662,0.150088,0.836575,1.398232,2.441186,0.785010,1.221262,0.726154,...,-0.089639,1.036171,1.092934,2.160679,1.477214,1.261134,1.475099,2.959403,1.555124,2.943482
3,2.002958,1.605841,2.581234,0.202040,0.870995,1.430943,2.490066,0.782768,1.185146,0.707268,...,-0.167862,1.034573,1.087285,2.176167,1.497215,1.271700,1.409464,2.914264,1.590504,2.940288
4,1.986856,1.585203,2.553937,0.221884,0.851413,1.438855,2.489432,0.773026,1.202064,0.697586,...,-0.132105,1.029349,1.091862,2.189674,1.506381,1.336551,1.386151,2.897875,1.581343,2.893509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,1.155123,1.901754,1.162571,0.910696,0.745226,0.835353,1.144480,0.652799,1.932211,0.842453,...,0.960492,1.787768,0.626726,2.011263,1.084712,1.249704,1.669961,2.061096,0.882343,1.957300
1444,1.152748,1.883088,1.184261,0.903087,0.751742,0.834111,1.189235,0.650191,1.972411,0.842808,...,0.950188,1.750430,0.624365,1.998901,1.088509,1.278237,1.637766,2.106696,0.902994,1.988022
1445,1.172412,1.908703,1.270773,0.916972,0.768153,0.839263,1.201479,0.652601,2.109775,0.914254,...,0.934080,1.702679,0.639688,1.992801,1.077241,1.324466,1.758190,2.190462,0.861626,2.049123
1446,1.172805,1.906460,1.270128,0.914495,0.766944,0.837897,1.198084,0.654865,2.110638,0.912225,...,0.928401,1.706282,0.640288,1.987775,1.090476,1.324441,1.760567,2.192025,0.859961,2.044745


In [242]:
df_weightz = secdata_cap_group.iloc[:, :].apply(lambda x: x.div(x.sum()), axis=1)
df_weightz

Ticker #,1,2,3,4,5,6,7,8,9,10,...,1868,1869,1870,1871,1872,1873,1874,1875,1876,1877
0,0.001326,0.003143,0.000100,0.000023,0.000291,0.000759,0.000031,0.000586,0.000015,0.000053,...,0.000008,0.000082,0.000029,0.000523,0.000023,0.001637,0.000031,0.000008,0.000094,0.000073
1,0.001345,0.003207,0.000099,0.000023,0.000285,0.000781,0.000029,0.000598,0.000015,0.000052,...,0.000007,0.000084,0.000029,0.000516,0.000024,0.001603,0.000031,0.000008,0.000094,0.000075
2,0.001385,0.003178,0.000105,0.000023,0.000289,0.000777,0.000032,0.000595,0.000016,0.000053,...,0.000007,0.000086,0.000029,0.000522,0.000025,0.001593,0.000031,0.000008,0.000094,0.000079
3,0.001399,0.003145,0.000108,0.000023,0.000297,0.000793,0.000033,0.000599,0.000016,0.000052,...,0.000007,0.000086,0.000029,0.000517,0.000025,0.001606,0.000031,0.000009,0.000093,0.000080
4,0.001456,0.003169,0.000100,0.000023,0.000288,0.000816,0.000034,0.000594,0.000016,0.000051,...,0.000007,0.000090,0.000031,0.000517,0.000025,0.001630,0.000032,0.000008,0.000093,0.000079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1506,0.000934,0.001421,0.000065,0.000031,0.000350,0.016806,0.000048,0.000677,0.000008,0.000041,...,0.000033,0.000038,0.000010,0.000159,0.000010,0.001146,0.000053,0.000029,0.000026,0.000052
1507,0.000936,0.001399,0.000062,0.000030,0.000348,0.016996,0.000049,0.000678,0.000009,0.000041,...,0.000033,0.000039,0.000010,0.000157,0.000010,0.001146,0.000053,0.000031,0.000025,0.000051
1508,0.000935,0.001395,0.000062,0.000031,0.000345,0.016821,0.000050,0.000679,0.000009,0.000042,...,0.000032,0.000039,0.000010,0.000157,0.000010,0.001145,0.000053,0.000030,0.000025,0.000051
1509,0.000959,0.001419,0.000062,0.000031,0.000347,0.017025,0.000050,0.000677,0.000009,0.000042,...,0.000032,0.000039,0.000010,0.000158,0.000010,0.001137,0.000052,0.000030,0.000024,0.000052


In [243]:
def market_beta(w, b):
    return np.dot(np.transpose(w), b)

### a)

#### Date: 12/30/2005 (idx loc = 503)

In [256]:
betas63_q3_2005 = []
for i in range(50):
    beta_m = betas_df_q3_63.iloc[63]
    weights = df_weightz[list(stocks.iloc[i])].iloc[63]
    betas_market = market_beta(weights, beta_m)
    betas63_q3_2005.append(betas_market)
betas63_q3_2005

[0.052476585476291554,
 0.060517796382025806,
 0.05266254554281719,
 0.04484296288982689,
 0.05008550819633388,
 0.11483905345383778,
 0.073322857947415,
 0.05505125994248011,
 0.12427471755206039,
 0.1163201164825795,
 0.0708626674840099,
 0.05541155423131752,
 0.05432297537972558,
 0.13442441034486619,
 0.10585230959008153,
 0.03577555334839695,
 0.09465118512066485,
 0.0714363304019478,
 0.03363909212684032,
 0.04195793292668462,
 0.04117789319176979,
 0.07031441366385044,
 0.05381480562401819,
 0.078138296288351,
 0.10093688542026702,
 0.051036879516628914,
 0.07667610383582535,
 0.06238527748300194,
 0.04332837367609722,
 0.03933712335535698,
 0.0724217109221468,
 0.061762543729987865,
 0.06345742579112622,
 0.07193427218599058,
 0.0694555719928294,
 0.07449283302234419,
 0.05083367356123818,
 0.06286017502275587,
 0.04995425185709235,
 0.047783139703163544,
 0.12594856391676224,
 0.06441863345230071,
 0.035499326887140074,
 0.04869310649040237,
 0.03921184699157464,
 0.0477617159

#### Date: 12/31/2007 (idx loc = 1005)

In [255]:
betas63_q3_2007 = []
for i in range(50):
    beta_m = betas_df_q3_63.iloc[942]
    weights = df_weightz[list(stocks.iloc[i])].iloc[942]
    betas_market = market_beta(weights, beta_m)
    betas63_q3_2007.append(betas_market)
betas63_q3_2007

[0.04844186126740678,
 0.05022385833208992,
 0.04548861529658407,
 0.05461929366595343,
 0.061548261018475034,
 0.06534954195811013,
 0.05688790637873255,
 0.046104948881788836,
 0.1076617144065238,
 0.11378194636284328,
 0.06277894880416302,
 0.05626732098236807,
 0.06218068742784019,
 0.13614324748653606,
 0.06585317014217026,
 0.03614911172060497,
 0.08520697207263117,
 0.06860030394451724,
 0.029300436262620907,
 0.045219318082515836,
 0.04961497148371338,
 0.05606900886658664,
 0.046407368801384026,
 0.06279147012171692,
 0.07378251846715525,
 0.06136129977648882,
 0.05525272969212655,
 0.05152475209489002,
 0.051416253736796196,
 0.03044853664343607,
 0.05956111460030072,
 0.05658907816646388,
 0.047019011027107045,
 0.05592551905126991,
 0.08339418247962005,
 0.07113678084787348,
 0.045472694165700295,
 0.06119284379865997,
 0.05623640029741284,
 0.04611064931309587,
 0.0688250843181559,
 0.06321828661968756,
 0.04204620124916704,
 0.05544096881555913,
 0.04857027940311408,
 0.0

#### Date: 12/31/2009 (idx loc = 1510)

In [258]:
betas63_q3_2009 = []
for i in range(50):
    beta_m = betas_df_q3_63.iloc[1447]
    weights = df_weightz[list(stocks.iloc[i])].iloc[1447]
    betas_market = market_beta(weights, beta_m)
    betas63_q3_2009.append(betas_market)
betas63_q3_2009

[0.041969788001929145,
 0.03609766003979558,
 0.0571643240845565,
 0.05099314794069944,
 0.06269323967843796,
 0.07007908597911089,
 0.06542687007033943,
 0.06294522474036876,
 0.1326222012094358,
 0.09869831219725504,
 0.08662925378641319,
 0.05210428234069958,
 0.07351577229722664,
 0.13157672809365403,
 0.06979837215389698,
 0.042045514913615294,
 0.07813386136379571,
 0.08461502709588789,
 0.02988254295835566,
 0.04835438020553477,
 0.05411589350054576,
 0.06816890698885693,
 0.045398859636565696,
 0.05664138297940627,
 0.0667691865821991,
 0.0686110393577253,
 0.06398265892236549,
 0.054928708373972485,
 0.06452772300637954,
 0.03939297329802155,
 0.06403339955640502,
 0.04670969799885384,
 0.054133111071517415,
 0.06748490015918884,
 0.08095751710068065,
 0.0926428927302992,
 0.04914394122850718,
 0.05888155868746275,
 0.07462290285725254,
 0.050435009620435664,
 0.08475775674172936,
 0.08500303631759625,
 0.04219611133616479,
 0.0776177220368038,
 0.05513947908324862,
 0.0457081

### b)

#### Date: 12/30/2005 (idx loc = 503)

In [293]:
#excludes the first 63 days
rp_2005 = 1/63*np.sum((dayahead63_port_ret_q3_arr[378:441] - np.array(ffdata['Risk-free rate'])[441:504]))
rp_2005

0.0010859704382670666

In [290]:
rm_2005 = 1/63*np.sum((np.array(ffdata['Market Returns'])[441:504] - np.array(ffdata['Risk-free rate'])[441:504]))
rm_2005

8.365079365079375e-05

#### Date: 12/31/2007 (idx loc = 1005)

In [286]:
rp_2007 = 1/63*np.sum((dayahead63_port_ret_q3_arr[880:943] - np.array(ffdata['Risk-free rate'])[943:1006]))
rp_2007

-0.00011254414214054921

In [291]:
rm_2007 = 1/63*np.sum((np.array(ffdata['Market Returns'])[943:1006] - np.array(ffdata['Risk-free rate'])[943:1006]))
rm_2007

-0.0008863492063492066

#### Date: 12/31/2009 (idx loc = 1510)

In [289]:
rp_2009 = 1/63*np.sum((dayahead63_port_ret_q3_arr[1385:1448] - np.array(ffdata['Risk-free rate'])[1448:1511]))
rp_2009

0.001998517909506608

In [292]:
rm_2009 = 1/63*np.sum((np.array(ffdata['Market Returns'])[1448:1511] - np.array(ffdata['Risk-free rate'])[1448:1511]))
rm_2009

0.0013793650793650792